---
# <center> **INVESTMENT GRADE FIXED INCOME ANALYSIS** </center>

----
# <center> **BUILDING A FIXED INCOME VALUATION PLATFORM** </center>

# Fixed Income Valuation Mini-Platform (Python)

## Objective
Build a production-grade fixed income valuation mini-platform in Python that mirrors the core workflow of an institutional valuation team pricing large corporate bond inventories daily.

The platform will demonstrate:
- Risk-free curve construction via bootstrapping (T-bills + Treasury notes)
- Fixed-rate bond pricing (clean/dirty, accrued interest, settlement)
- Corporate bond pricing using spread-over-curve discounting
- Risk measures: duration, modified duration, convexity, DV01, spread duration
- Scenario analysis: rate shocks (parallel + twists) and spread shocks
- Vectorized portfolio pricing (20+ bonds) with quality control and validation
- Modular, production-style architecture and interview-ready explanations

This project is designed to be defensible in a Pricing Direct Evaluator / Fixed Income Valuation interview context.

---

## Core Modeling Conventions 
We will explicitly define conventions because pricing is convention-sensitive.

### Time and Discounting
We represent the curve primarily via discount factors $D(t)$.
For a continuously compounded zero rate $z(t)$ and year fraction $\tau(t)$:

$$
D(t) = e^{-z(t)\tau(t)}
$$

We bootstrap discount factors at instrument maturities and interpolate between knots.

### Bills (Money-Market Simple Yield)
Given a bill quote $y$ (simple yield) and year fraction $\tau$:

$$
D(T) = \frac{1}{1 + y\tau}
$$

### Notes (Par Yield Bootstrapping)
For a Treasury note with par yield $c$, coupon frequency $m=2$,
coupon per period $\frac{c}{m}$, and cashflow dates $t_1,\dots,t_n$,
par pricing implies:

$$
1 = \sum_{i=1}^{n}\frac{c}{m}D(t_i) + D(t_n)
$$

Solving for the final discount factor:

$$
D(t_n) = \frac{1 - \sum_{i=1}^{n-1}\frac{c}{m}D(t_i)}{1 + \frac{c}{m}}
$$

### Interpolation Choice (Default)
Default interpolation: linear interpolation of $\log D(t)$.

Reason:
- Guarantees strictly positive discount factors
- More stable than interpolating yields directly
- Reduces overshoot risk relative to unconstrained cubic splines

---

# Fixed Income Valuation Mini-Platform

## Project Objective

This project builds a fixed income valuation and risk analytics engine in Python. The objective is not merely to price bonds, but to construct a complete valuation infrastructure capable of:

- Yield curve construction
- Clean and dirty bond pricing
- Corporate spread modeling
- Sensitivity analysis (DV01, convexity, key rate duration)
- Spread risk measurement
- Scenario stress testing
- Portfolio-level risk aggregation
- Internal consistency validation

---

## Why This Project Matters

In institutional environments (investment banks, asset managers, risk teams), fixed income valuation requires:

1. Arbitrage-free yield curve construction  
2. Settlement-aware pricing mechanics  
3. Accurate accrued interest and clean/dirty decomposition  
4. Robust sensitivity calculations  
5. Scenario-based stress testing  
6. Reconciliation between risk metrics and scenario results  
7. Scalability to large bond portfolios  

This project implements all of the above in a modular and production-oriented manner. The mini valuation engine built in this project could be extended to price millions of bonds daily.

---

## What Has Been Built

### 1. Yield Curve Construction

- Bootstrapped zero-coupon yield curve from Treasury bills and notes
- Log-linear interpolation in discount factor space
- Monotonic discount factor validation
- Continuously compounded zero rate extraction
- Settlement-adjusted discounting

This ensures arbitrage-free and stable curve construction.

---

### 2. Bond Pricing Engine

- Fixed-rate semiannual coupon bonds
- Clean and dirty pricing
- Accrued interest under 30/360 and ACT conventions
- Settlement-adjusted discount factors
- Vectorized pricing across portfolios

Pricing follows:

$$
P = \sum_i CF_i \cdot \frac{D(t_i)}{D(s)}
$$

where $s$ is settlement date.

---

### 3. Risk Measures

- Parallel DV01 (finite difference)
- Convexity (second-order finite difference)
- Linear vs nonlinear validation
- Key Rate Duration (KRD) using partition-of-unity hat basis functions
- Reconciliation of KRD sum to parallel DV01

This ensures internal risk consistency:

$$
\sum_k KRD_k \approx DV01_{\text{parallel}}
$$

---

### 4. Corporate Bond Extension

- Constant Z-spread applied to **all cashflows**
- Spread-adjusted discounting:

$$
D_{corp}(t_i) = \frac{D(t_i)}{D(s)} \cdot e^{-s \cdot \tau(s, t_i)}
$$

- Spread DV01
- Spread duration
- Spread shock scenarios

This models credit risk consistently across the entire cashflow stream.

---

### 5. Scenario Analysis

- Parallel rate shocks (±25bp, ±50bp)
- Steepener and flattener curve shocks
- Spread widening scenarios
- Combined rate + spread stress grid

Scenario P&L reconciles with sensitivity approximations, validating implementation correctness.

---

### 6. Portfolio Risk Dashboard

- Total market value
- Parallel DV01
- Spread DV01
- Key rate bucket decomposition
- Concentration analysis (top risk contributors)
- Scenario impact summaries

---

## Relevance to Institutional Valuation Teams

This project directly demonstrates competencies required in:

- Pricing Direct Evaluation
- Market Risk Analytics
- Model Validation
- Fixed Income Quantitative Research
- Valuation Control

It shows understanding of:

- Discount curve mechanics
- Credit spread modeling
- Risk reconciliation
- Sensitivity validation
- Term structure decomposition
- Stress scenario interpretation

Most importantly, it demonstrates the ability to design valuation infrastructure — not just compute prices.

---

## Design Philosophy

The platform was built with production principles in mind:

- Modular architecture
- Clear separation of curve, pricing, risk, and scenario logic
- Vectorized portfolio computation
- Defensive data validation
- Reconciliation diagnostics

The result is a scalable valuation framework that can be extended to:

- OAS modeling
- Stochastic interest rate models
- Large-scale portfolio processing
- Regulatory risk reporting

# **I. Yield Curve Construction (Bootstrapping)**

## Goal
Bootstrap a risk-free discount curve from:
- Treasury bills (money-market simple yields)
- Treasury notes (par yields, semiannual coupons)

We will:
1. Bootstrap discount factors $D(t)$ at instrument maturities ("knots")
2. Interpolate $\log D(t)$ between knots
3. Derive zero rates $z(t)$ as needed
4. Wrap results in a reusable curve object with QC checks

---

## Conventions (Default)
- Bills: ACT/360 for money-market year fraction in $D(T)=\frac{1}{1+y\tau}$
- Notes: 30/360 for coupon accrual fractions
- Curve time for continuous zeros: ACT/365
- Interpolation: linear interpolation on $\log D(t)$

---

## Bootstrapping Theory (The Math)

### Bills (simple yield)
Given quote $y$ and year fraction $\tau$:
$$
D(T)=\frac{1}{1+y\tau}
$$

### Notes (par yields)
For maturity $t_n$, par yield $c$, frequency $m=2$, coupon $\frac{c}{m}$:
$$
1=\sum_{i=1}^{n}\frac{c}{m}D(t_i)+D(t_n)
$$

Solve for $D(t_n)$:
$$
D(t_n)=\frac{1-\sum_{i=1}^{n-1}\frac{c}{m}D(t_i)}{1+\frac{c}{m}}
$$

We will compute $D(t_i)$ for intermediate coupon dates using interpolation on $\log D(t)$ from already-bootstrapped knots.


In [1]:
# Imports and Necessary libraries

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Optional, Iterable
from scipy.optimize import brentq

In [2]:
# Day count utilities

def yearfrac(start: pd.Timestamp, end: pd.Timestamp, convention: str) -> float:
    """
    Year fraction between two dates under a day count convention.

    Supported:
    - ACT/365, ACT/365F
    - ACT/360
    - 30/360, 30/360US (US bond basis)
    """
    convention = convention.upper().replace(" ", "")
    if end < start:
        raise ValueError(f"end < start: {start=} {end=}")

    if convention in ("ACT/365", "ACT/365F"):
        return (end - start).days / 365.0

    if convention == "ACT/360":
        return (end - start).days / 360.0

    if convention in ("30/360", "30/360US"):
        y1, m1, d1 = start.year, start.month, start.day
        y2, m2, d2 = end.year, end.month, end.day

        # 30/360 US convention
        if d1 == 31:
            d1 = 30
        if d2 == 31 and d1 == 30:
            d2 = 30

        return ((y2 - y1) * 360 + (m2 - m1) * 30 + (d2 - d1)) / 360.0

    raise ValueError(f"Unsupported day count convention: {convention}")

In [3]:
# Schedule generation

def semiannual_coupon_dates(val_date: pd.Timestamp, maturity: pd.Timestamp) -> List[pd.Timestamp]:
    """
    Generate semiannual coupon payment dates strictly AFTER val_date, ending at maturity.

    This avoids accidentally including a coupon date that is only a couple days after val_date
    due to naive backward stepping.
    """
    val_date = pd.Timestamp(val_date)
    maturity = pd.Timestamp(maturity)

    if maturity <= val_date:
        raise ValueError("Maturity must be after valuation date.")

    # Step backward in 6M increments to find the last coupon date <= val_date
    d = maturity
    while d > val_date:
        d = d - pd.DateOffset(months=6)

    prev_coupon = d  # <= val_date
    next_coupon = prev_coupon + pd.DateOffset(months=6)

    # Build forward schedule from next_coupon to maturity
    dates = []
    d = next_coupon
    while d < maturity:
        dates.append(d)
        d = d + pd.DateOffset(months=6)

    dates.append(maturity)
    # Ensure strictly greater than val_date
    dates = [x for x in dates if x > val_date]

    return dates

In [21]:
# Curve object (interpolation on log DF)
## vectorized DF queries + caching.

@dataclass(frozen=True)
class ZeroCurve:
    """
    Discount curve represented by bootstrapped knot discount factors,
    interpolated linearly in log discount factor space.
    """
    val_date: pd.Timestamp
    knot_dates: np.ndarray          # dtype datetime64[ns]
    knot_log_dfs: np.ndarray        # log(D)
    zero_day_count: str = "ACT/365"

    def _to_datetime64(self, dates: Iterable[pd.Timestamp]) -> np.ndarray:
        return np.array([pd.Timestamp(d).to_datetime64() for d in dates], dtype="datetime64[ns]")

    def df(self, dates: Iterable[pd.Timestamp]) -> np.ndarray:
        """
        Vectorized discount factor lookup with log-linear interpolation.
        Short-end extrapolation (between val_date and first knot) uses flat
        continuously-compounded zero rate implied by the first knot.
        """
        dates_list = [pd.Timestamp(d) for d in dates]
        x = self._to_datetime64(dates_list).astype("datetime64[ns]").astype("int64")
        kx = self.knot_dates.astype("datetime64[ns]").astype("int64")
        kv = self.knot_log_dfs
    
        x_min, x_max = x.min(), x.max()
        k_min, k_max = kx.min(), kx.max()
    
        if x_max > k_max:
            raise ValueError("Requested date beyond curve knot range (no long-end extrapolation).")
    
        # Precompute first-knot flat cc zero for short-end extrapolation
        first_date = pd.Timestamp(pd.to_datetime(self.knot_dates[0]))
        df1 = float(np.exp(kv[0]))
        tau1 = yearfrac(self.val_date, first_date, self.zero_day_count)
        if tau1 <= 0:
            raise ValueError("First knot must be after valuation date.")
        z1 = -np.log(df1) / tau1
    
        out = np.empty_like(x, dtype=float)
    
        # Short-end: x < first knot => D(t)=exp(-z1*tau(t))
        mask_short = x < k_min
        if np.any(mask_short):
            taus = np.array([yearfrac(self.val_date, dates_list[i], self.zero_day_count)
                             for i in np.where(mask_short)[0]], dtype=float)
            if np.any(taus < 0):
                raise ValueError("Requested date before valuation date.")
            out[mask_short] = np.exp(-z1 * taus)
    
        # Within knots: log-linear interpolation
        mask_in = ~mask_short
        if np.any(mask_in):
            log_df = np.interp(x[mask_in], kx, kv)
            out[mask_in] = np.exp(log_df)

        return out


    def zero_rate_cc(self, dates: Iterable[pd.Timestamp]) -> np.ndarray:
        """
        Continuously compounded zero rate z(t) implied by D(t)=exp(-z*tau).
        tau uses zero_day_count (default ACT/365).
        """
        dates_list = [pd.Timestamp(d) for d in dates]
        dfs = self.df(dates_list)

        taus = np.array([yearfrac(self.val_date, d, self.zero_day_count) for d in dates_list], dtype=float)
        if np.any(taus <= 0):
            raise ValueError("Non-positive tau encountered in zero rate computation.")

        return -np.log(dfs) / taus

## Market Data Schema

We represent market instruments in a DataFrame with realistic fields:

### Bills
- type = "bill"
- maturity (date)
- quote = simple yield (annualized)
- day_count = "ACT/360" (market)

### Notes
- type = "note"
- maturity (date)
- quote = par yield (annualized)
- coupon_freq = 2
- day_count = "30/360" (bond basis)

We bootstrap discount factors in maturity order.

In [22]:
# Example market data (simulated but realistic structure)

val_date = pd.Timestamp("2026-02-13")

market = pd.DataFrame([
    {"type": "bill", "maturity": pd.Timestamp("2026-03-13"), "quote": 0.0520, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-05-15"), "quote": 0.0515, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-08-14"), "quote": 0.0505, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2027-02-12"), "quote": 0.0485, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-02-20"), "quote": 0.0525, "day_count": "ACT/360"},
    {"type": "note", "maturity": pd.Timestamp("2028-02-15"), "quote": 0.0450, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2031-02-15"), "quote": 0.0430, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2036-02-15"), "quote": 0.0425, "coupon_freq": 2, "day_count": "30/360"},
])

market = market.sort_values("maturity").reset_index(drop=True)
market

,type,maturity,quote,day_count,coupon_freq
0,bill,2026-02-20,0.0525,ACT/360,NaN
1,bill,2026-03-13,0.0520,ACT/360,NaN
2,bill,2026-05-15,0.0515,ACT/360,NaN
3,bill,2026-08-14,0.0505,ACT/360,NaN
4,bill,2027-02-12,0.0485,ACT/360,NaN
5,note,2028-02-15,0.0450,30/360,2.0
6,note,2031-02-15,0.0430,30/360,2.0
7,note,2036-02-15,0.0425,30/360,2.0


In [23]:
# Bootstrapping engine (bootstrap knots, update curve, proceed)

def bootstrap_curve_from_bills_notes(
    market: pd.DataFrame,
    val_date: pd.Timestamp,
    zero_day_count: str = "ACT/365",
    enforce_monotone_df: bool = True,
) -> ZeroCurve:

    market = market.sort_values("maturity").reset_index(drop=True)

    # Maintain knots as dict: {date: logDF}
    knot_logdf: dict[pd.Timestamp, float] = {}

    def _sorted_knots():
        dates = sorted(knot_logdf.keys())
        logdfs = np.array([knot_logdf[d] for d in dates], dtype=float)
        kx = np.array(
            [d.to_datetime64().astype("datetime64[ns]").astype("int64") for d in dates],
            dtype=np.int64,
        )
        return dates, logdfs, kx

    def current_curve() -> Optional[ZeroCurve]:
        dates, logdfs, _ = _sorted_knots()
        if len(dates) < 2:
            return None
        kd = np.array([d.to_datetime64() for d in dates], dtype="datetime64[ns]")
        return ZeroCurve(val_date, kd, logdfs, zero_day_count)

    # =====================================================
    # STEP A: Bills
    # =====================================================
    for _, row in market[market["type"] == "bill"].iterrows():
        T = pd.Timestamp(row["maturity"])
        y = float(row["quote"])
        dc = str(row.get("day_count", "ACT/360"))

        tau = yearfrac(val_date, T, dc)
        if tau <= 0:
            raise ValueError("Bill maturity must be after valuation date.")

        df_T = 1.0 / (1.0 + y * tau)
        if not (0.0 < df_T < 1.5):
            raise ValueError("Bill DF out of bounds.")

        knot_logdf[T] = float(np.log(df_T))

        if enforce_monotone_df:
            dates, logdfs, _ = _sorted_knots()
            if np.any(np.diff(np.exp(logdfs)) > 1e-10):
                raise ValueError("Non-monotone DF detected in bills.")

    if len(knot_logdf) < 2:
        raise ValueError("Need at least two bills before notes.")

    # =====================================================
    # STEP B: Notes (root solve)
    # =====================================================
    for _, row in market[market["type"] == "note"].iterrows():

        T = pd.Timestamp(row["maturity"])
        c = float(row["quote"])
        freq = int(row.get("coupon_freq", 2))
        if freq != 2:
            raise NotImplementedError("Only semiannual supported.")

        curve = current_curve()
        if curve is None:
            raise ValueError("Curve not initialized.")

        cpn_dates = semiannual_coupon_dates(val_date, T)
        coupon = c / freq

        dates, logdfs, kx = _sorted_knots()
        kv = logdfs

        tL = dates[-1]
        if tL >= T:
            raise ValueError("Note maturity must exceed last knot.")

        logDF_L = knot_logdf[tL]

        tL_int = tL.to_datetime64().astype("datetime64[ns]").astype("int64")
        T_int = T.to_datetime64().astype("datetime64[ns]").astype("int64")

        def df_with_unknown_endpoint(date: pd.Timestamp, logDF_T: float) -> float:

            x = date.to_datetime64().astype("datetime64[ns]").astype("int64")

            # -------------------------
            # Case 1: short-end extrapolation
            # -------------------------
            if x < kx.min():
                first_date = dates[0]
                first_logdf = kv[0]
                df1 = float(np.exp(first_logdf))

                tau1 = yearfrac(val_date, first_date, zero_day_count)
                z1 = -np.log(df1) / tau1

                tau_x = yearfrac(val_date, date, zero_day_count)
                return float(np.exp(-z1 * tau_x))

            # -------------------------
            # Case 2: within known knots
            # -------------------------
            if x <= tL_int:
                logDF = float(np.interp(x, kx, kv))
                return float(np.exp(logDF))

            # -------------------------
            # Case 3: between (tL, T]
            # -------------------------
            if x <= T_int:
                w = (x - tL_int) / (T_int - tL_int)
                logDF = (1 - w) * logDF_L + w * logDF_T
                return float(np.exp(logDF))

            raise ValueError("Date beyond maturity during bootstrap.")

        def par_residual(logDF_T: float) -> float:
            pv = 0.0
            for d in cpn_dates:
                pv += coupon * df_with_unknown_endpoint(d, logDF_T)
            pv += df_with_unknown_endpoint(T, logDF_T)
            return pv - 1.0

        # Solve
        a = np.log(1e-6)
        b = min(logDF_L, -1e-12)

        fa, fb = par_residual(a), par_residual(b)
        if fa * fb > 0:
            raise ValueError("Root not bracketed — inconsistent market data.")

        logDF_T = brentq(par_residual, a, b, maxiter=300, xtol=1e-14)

        knot_logdf[T] = float(logDF_T)

        if enforce_monotone_df:
            dates2, logdfs2, _ = _sorted_knots()
            if np.any(np.diff(np.exp(logdfs2)) > 1e-10):
                raise ValueError("Non-monotone DF after adding note.")

    # =====================================================
    # Final curve
    # =====================================================
    final_dates, final_logdfs, _ = _sorted_knots()
    kd = np.array([d.to_datetime64() for d in final_dates], dtype="datetime64[ns]")

    return ZeroCurve(val_date, kd, final_logdfs, zero_day_count)

In [24]:
# QC diagnostics

def curve_qc_report(curve: ZeroCurve) -> pd.DataFrame:
    dates = pd.to_datetime(curve.knot_dates)
    dfs = np.exp(curve.knot_log_dfs)

    taus = np.array([
        yearfrac(curve.val_date, pd.Timestamp(d), curve.zero_day_count)
        for d in dates
    ])

    zeros = -np.log(dfs) / taus

    return pd.DataFrame({
        "date": dates,
        "tau": taus,
        "df": dfs,
        "zero_cc": zeros,
        "df_positive": dfs > 0,
        "df_monotone": np.r_[True, np.diff(dfs) <= 1e-10],
    })


In [25]:
# Build curve + inspect knots

curve = bootstrap_curve_from_bills_notes(market, val_date)

knot_df = pd.DataFrame({
    "knot_date": pd.to_datetime(curve.knot_dates),
    "df": np.exp(curve.knot_log_dfs),
})
print(knot_df)

qc = curve_qc_report(curve)
display(qc.head(10))
display(qc.tail(10))

assert (np.exp(curve.knot_log_dfs) > 0).all()
assert (np.diff(np.exp(curve.knot_log_dfs)) <= 1e-10).all(), "DF not monotone decreasing"


   knot_date        df
0 2026-02-20  0.998980
1 2026-03-13  0.995972
2 2026-05-15  0.987149
3 2026-08-14  0.975105
4 2027-02-12  0.953254
5 2028-02-15  0.893272
6 2031-02-15  0.790633
7 2036-02-15  0.642857


,date,tau,df,zero_cc,df_positive,df_monotone
0,2026-02-20,0.019178,0.998980,0.053202,True,True
1,2026-03-13,0.076712,0.995972,0.052616,True,True
2,2026-05-15,0.249315,0.987149,0.051878,True,True
3,2026-08-14,0.498630,0.975105,0.050559,True,True
4,2027-02-12,0.997260,0.953254,0.048006,True,True
5,2028-02-15,2.005479,0.893272,0.056278,True,True
6,2031-02-15,5.008219,0.790633,0.046907,True,True
7,2036-02-15,10.010959,0.642857,0.044135,True,True


,date,tau,df,zero_cc,df_positive,df_monotone
0,2026-02-20,0.019178,0.998980,0.053202,True,True
1,2026-03-13,0.076712,0.995972,0.052616,True,True
2,2026-05-15,0.249315,0.987149,0.051878,True,True
3,2026-08-14,0.498630,0.975105,0.050559,True,True
4,2027-02-12,0.997260,0.953254,0.048006,True,True
5,2028-02-15,2.005479,0.893272,0.056278,True,True
6,2031-02-15,5.008219,0.790633,0.046907,True,True
7,2036-02-15,10.010959,0.642857,0.044135,True,True


In [28]:
# Repricing the 2028 note using the curve and verifing it prices to par.

def price_par_note_check(curve, maturity, par_yield):
    freq = 2
    coupon = par_yield / freq
    cpn_dates = semiannual_coupon_dates(val_date, maturity)

    pv = 0.0
    for d in cpn_dates:
        pv += coupon * curve.df([d])[0]
    pv += curve.df([maturity])[0]

    return pv

print(semiannual_coupon_dates(val_date, pd.Timestamp("2028-02-15")))
print(price_par_note_check(curve, pd.Timestamp("2028-02-15"), 0.0450))


[Timestamp('2026-02-15 00:00:00'), Timestamp('2026-08-15 00:00:00'), Timestamp('2027-02-15 00:00:00'), Timestamp('2027-08-15 00:00:00'), Timestamp('2028-02-15 00:00:00')]
0.9999999999999999


---

## Yield Curve Boostrapping Results: Bootstrapped Risk-Free Curve (Bills + Notes)

We successfully constructed a reusable discount curve from:
- Treasury bills quoted as simple yields (ACT/360)
- Treasury notes quoted as par yields (semiannual coupons)

### Key design decisions
- Curve representation: discount factors $D(t)$ at bootstrapped knot maturities.
- Interpolation: linear in $\log D(t)$ to ensure $D(t) > 0$ and stabilize numerical behavior.
- Short-end extrapolation (between valuation date and first knot): flat continuously-compounded zero rate implied by the first knot:
$$
z_1 = -\frac{\ln D(t_1)}{\tau(t_1)}, \quad D(t)=e^{-z_1 \tau(t)} \;\; \text{for} \;\; t \le t_1.
$$
- Notes are calibrated by solving for $\log D(T)$ via root-finding so that par pricing holds.

### Internal consistency check
Repricing the 2Y par note using the bootstrapped curve returned $PV \approx 1.0$, confirming that the curve calibration and curve discounting logic are consistent.

---

# **II. Bond Pricing Engine (Accrued Interest, Settlement)**

## Goal
Implement a production-style fixed-rate bond pricing engine that:
- Constructs coupon cashflows from a schedule
- Discounts cashflows using the bootstrapped curve $D(t)$
- Computes accrued interest using market day-count conventions
- Returns dirty and clean prices
- Supports semiannual coupon bonds (baseline IG convention)
- Provides a vectorized pathway for portfolio pricing

---

## Core Definitions

### Settlement vs Valuation Date
- **Valuation date** $t_0$: the market snapshot date (curve build date).
- **Settlement date** $t_{set}$: the date the bond trade settles (typically $t_0 + 2$ business days for corporates; simplified as $t_0 + 2$ calendar days for now).

Pricing and accrued interest are computed as of $t_{set}$.

---

## Cashflows

For a bond with face $N$, annual coupon rate $c$, coupon frequency $m$ (semiannual: $m=2$),
coupon payment dates $t_1,\dots,t_n$ after settlement:

Coupon cashflow at date $t_i$:
- Simplified regular schedule (baseline): $CF^{coupon}_i = N \cdot \frac{c}{m}$
- More general (to be used for irregular/stub periods): $CF^{coupon}_i = N \cdot c \cdot \alpha_i$ where $\alpha_i$ is the accrual fraction.

Principal at maturity $T$:
$$
CF^{principal}(T) = N
$$

---

## Discounting (Dirty Price)

Dirty price (per 100 notional) at settlement:
$$
P_{dirty} = \frac{100}{N}\sum_{i=1}^{n} CF_i \cdot D(t_i)
$$

We use the curve’s discount factor function $D(t)$ with the curve’s interpolation/extrapolation policy.

---

## Accrued Interest

Accrued interest at settlement depends on day count convention (e.g., 30/360, ACT/365).

Let:
- $t_{prev}$ = previous coupon date (on or before settlement)
- $t_{next}$ = next coupon date (after settlement)

For regular coupon schedules:
$$
AI = N \cdot \frac{c}{m}\cdot \frac{\text{yearfrac}(t_{prev}, t_{set})}{\text{yearfrac}(t_{prev}, t_{next})}
$$

---

## Clean Price

Clean price is the quoted market price:
$$
P_{clean} = P_{dirty} - \frac{100}{N}AI
$$

---

## QC / Validation Checks (Production Mindset)
Before pricing we validate:
- $t_{set} < T$ (not matured)
- coupon frequency is supported
- coupon rate is reasonable ($c \ge 0$ typically for IG)
- schedule dates are strictly increasing
- day count convention recognized


In [29]:
# Imports and Helpers

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List, Optional, Iterable, Tuple


In [30]:
# Settlement convention (T+2 baseline)

def settlement_date(val_date: pd.Timestamp, lag_days: int = 2) -> pd.Timestamp:
    """
    Simplified settlement date: valuation date + lag_days (calendar days).
    In production: use business-day calendars + holiday schedules.
    """
    return pd.Timestamp(val_date) + pd.Timedelta(days=lag_days)

In [31]:
# Coupon date utilities (previous/next coupon)

def previous_coupon_date(settle: pd.Timestamp, maturity: pd.Timestamp, freq: int = 2) -> pd.Timestamp:
    """
    Find the most recent coupon date on or before settlement, using a schedule
    anchored at maturity and stepping backward by 12/freq months.

    This is a pragmatic approximation when issue date is unknown.
    """
    if freq <= 0:
        raise ValueError("freq must be positive")
    months = int(12 / freq)

    d = pd.Timestamp(maturity)
    settle = pd.Timestamp(settle)

    # Step backwards until d <= settle
    while d > settle:
        d = d - pd.DateOffset(months=months)
    return d

def next_coupon_date(settle: pd.Timestamp, maturity: pd.Timestamp, freq: int = 2) -> pd.Timestamp:
    """
    Next coupon date strictly after settlement.
    """
    prev = previous_coupon_date(settle, maturity, freq)
    months = int(12 / freq)
    nxt = prev + pd.DateOffset(months=months)
    return nxt

In [32]:
# Cashflow schedule after settlement

def coupon_schedule_after_settlement(
    settle: pd.Timestamp,
    maturity: pd.Timestamp,
    freq: int = 2,
) -> List[pd.Timestamp]:
    """
    All coupon payment dates strictly after settlement, ending at maturity.
    """
    if settle >= maturity:
        return []

    months = int(12 / freq)
    d = next_coupon_date(settle, maturity, freq)

    dates = []
    while d < maturity:
        dates.append(d)
        d = d + pd.DateOffset(months=months)

    dates.append(pd.Timestamp(maturity))
    return dates

In [33]:
# Accrued interest

def accrued_interest(
    settle: pd.Timestamp,
    maturity: pd.Timestamp,
    coupon_rate: float,
    face: float,
    freq: int,
    day_count: str,
) -> float:
    """
    Accrued interest amount in currency units (not per 100).
    """
    if settle >= maturity:
        return 0.0

    t_prev = previous_coupon_date(settle, maturity, freq)
    t_next = next_coupon_date(settle, maturity, freq)

    # Accrued fraction of the coupon period
    accrual_num = yearfrac(t_prev, settle, day_count)
    accrual_den = yearfrac(t_prev, t_next, day_count)

    if accrual_den <= 0:
        raise ValueError("Invalid coupon period length from schedule/daycount.")

    coupon_per_period = face * (coupon_rate / freq)
    return coupon_per_period * (accrual_num / accrual_den)

In [34]:
# Bond definition

@dataclass(frozen=True)
class Bond:
    bond_id: str
    maturity: pd.Timestamp
    coupon_rate: float          # annual coupon rate, e.g. 0.05 for 5%
    freq: int = 2               # semiannual default
    day_count: str = "30/360"   # typical for corporates
    face: float = 100.0         # notional

In [42]:
# Pricing: dirty and clean

def price_bond_dirty_clean(
    curve: ZeroCurve,
    bond: Bond,
    val_date: pd.Timestamp,
    settle: Optional[pd.Timestamp] = None,
) -> Tuple[float, float, float]:
    """
    Returns (dirty_price_per_100, clean_price_per_100, accrued_interest_per_100).

    Dirty = PV of remaining cashflows / face * 100
    Clean = Dirty - Accrued
    """
    if settle is None:
        settle = settlement_date(val_date, lag_days=2)

    settle = pd.Timestamp(settle)
    maturity = pd.Timestamp(bond.maturity)

    if settle >= maturity:
        raise ValueError(f"Bond {bond.bond_id} is matured or settles on/after maturity.")

    if bond.freq not in (1, 2, 4):
        raise NotImplementedError("Supported frequencies: 1, 2, 4 (for now).")

    if bond.coupon_rate < -0.01:
        raise ValueError("Suspiciously negative coupon rate.")

    # Cashflow dates after settlement
    pay_dates = coupon_schedule_after_settlement(settle, maturity, bond.freq)
    if len(pay_dates) == 0:
        raise ValueError("No remaining cashflows after settlement; check schedule logic.")

    # Coupon CF (regular schedule assumption)
    coupon_cf = bond.face * (bond.coupon_rate / bond.freq)

    # Build cashflows vector
    cfs = np.full(len(pay_dates), coupon_cf, dtype=float)
    cfs[-1] += bond.face  # principal at maturity

    # Discount factors for payment dates
    dfs_pay = curve.df(pay_dates)
    df_settle = float(curve.df([settle])[0])
    
    # settlement-adjusted discount factors: D_s(T)=D(T)/D(s)
    dfs_settle_adj = dfs_pay / df_settle
    
    pv = float(np.sum(cfs * dfs_settle_adj))
    dirty_per_100 = 100.0 * pv / bond.face


    ai = accrued_interest(settle, maturity, bond.coupon_rate, bond.face, bond.freq, bond.day_count)
    ai_per_100 = 100.0 * ai / bond.face

    clean_per_100 = dirty_per_100 - ai_per_100

    return dirty_per_100, clean_per_100, ai_per_100

---

## Sanity Checks

We test:
1. Pricing returns finite values
2. Dirty price > clean price when accrued interest > 0
3. If settlement is very close to a coupon date, accrued interest behaves as expected

In [43]:
# Demo Bond Pricing

# Example bond (IG-style)
bond = Bond(
    bond_id="CORP_5Y_5PCT",
    maturity=pd.Timestamp("2031-02-15"),
    coupon_rate=0.05,
    freq=2,
    day_count="30/360",
    face=100.0,
)

val_date = pd.Timestamp("2026-02-13")
settle = settlement_date(val_date, 2)

dirty, clean, ai = price_bond_dirty_clean(curve, bond, val_date, settle)

print("Settlement:", settle.date())
print("Dirty price:", dirty)
print("Accrued (per 100):", ai)
print("Clean price:", clean)

assert np.isfinite(dirty) and np.isfinite(clean) and np.isfinite(ai)
assert dirty >= clean - 1e-10

Settlement: 2026-02-15
Dirty price: 100.93844252553204
Accrued (per 100): 0.0
Clean price: 100.93844252553204


In [44]:
print(coupon_schedule_after_settlement(settle, bond.maturity, bond.freq)[:6], "...", "last:", coupon_schedule_after_settlement(settle, bond.maturity, bond.freq)[-1])

[Timestamp('2026-08-15 00:00:00'), Timestamp('2027-02-15 00:00:00'), Timestamp('2027-08-15 00:00:00'), Timestamp('2028-02-15 00:00:00'), Timestamp('2028-08-15 00:00:00'), Timestamp('2029-02-15 00:00:00')] ... last: 2031-02-15 00:00:00


---

## Bond Pricing Validation Layer (Why a valuation team cares)

In production, most errors are not numerical - they are input and convention errors.
We therefore add validation checks and "reasonableness flags" before scaling.

### Checks we enforce
- Settlement < maturity (not matured)
- Coupon rate is within a plausible range (e.g., $0\%$ to $20\%$ for IG)
- Schedule dates strictly increasing
- Accrued interest satisfies:
  - $0 \le AI \le$ coupon per period (for regular schedules)
- Dirty price and clean price are finite
- Dirty price $\ge$ clean price (when $AI \ge 0$)

These checks prevent silent corruption when pricing portfolios.

In [45]:
# Accrued interest should jump from ~coupon to 0 at coupon date, depending on convention.
test_settles = [
    pd.Timestamp("2026-02-14"),
    pd.Timestamp("2026-02-15"),  # coupon date
    pd.Timestamp("2026-02-16"),
]

for s in test_settles:
    dirty, clean, ai = price_bond_dirty_clean(curve, bond, val_date, s)
    print(s.date(), "AI:", ai, "Dirty:", dirty, "Clean:", clean)
    assert dirty >= clean - 1e-10
    assert ai >= -1e-12

2026-02-14 AI: 2.486111111111111 Dirty: 103.42336654570346 Clean: 100.93725543459234
2026-02-15 AI: 0.0 Dirty: 100.93844252553204 Clean: 100.93844252553204
2026-02-16 AI: 0.01388888888888889 Dirty: 100.95315627905092 Clean: 100.93926739016203


In [46]:
# Coupon period reasonableness check

def qc_bond_schedule(settle: pd.Timestamp, maturity: pd.Timestamp, freq: int) -> None:
    dates = coupon_schedule_after_settlement(settle, maturity, freq)
    if any(dates[i] >= dates[i+1] for i in range(len(dates)-1)):
        raise ValueError("Schedule is not strictly increasing.")
    if dates[-1] != pd.Timestamp(maturity):
        raise ValueError("Schedule does not end at maturity.")

In [47]:
qc_bond_schedule(settle, bond.maturity, bond.freq)

In [48]:
# Compare ACT/365 vs 30/360 accrued (same bond, different daycount)

bond_30360 = Bond("BOND_30360", bond.maturity, bond.coupon_rate, bond.freq, "30/360", bond.face)
bond_act365 = Bond("BOND_ACT365", bond.maturity, bond.coupon_rate, bond.freq, "ACT/365", bond.face)

s = pd.Timestamp("2026-02-16")  # one day after coupon date
_, _, ai_30360 = price_bond_dirty_clean(curve, bond_30360, val_date, s)
_, _, ai_act365 = price_bond_dirty_clean(curve, bond_act365, val_date, s)

print("AI 30/360:", ai_30360, "AI ACT/365:", ai_act365)

AI 30/360: 0.01388888888888889 AI ACT/365: 0.013812154696132596


In [49]:
class BondPricer:
    def __init__(self, curve: ZeroCurve):
        self.curve = curve

    def validate(self, bond: Bond, settle: pd.Timestamp) -> None:
        if settle >= bond.maturity:
            raise ValueError(f"{bond.bond_id}: matured at settlement.")
        if bond.freq not in (1, 2, 4):
            raise NotImplementedError("Supported frequencies: 1, 2, 4.")
        if not (-0.01 <= bond.coupon_rate <= 0.25):
            raise ValueError(f"{bond.bond_id}: coupon out of plausible range.")
        # schedule monotonicity
        dates = coupon_schedule_after_settlement(settle, bond.maturity, bond.freq)
        if any(dates[i] >= dates[i+1] for i in range(len(dates)-1)):
            raise ValueError(f"{bond.bond_id}: non-increasing schedule.")

    def price(self, bond: Bond, val_date: pd.Timestamp, settle: Optional[pd.Timestamp] = None):
        if settle is None:
            settle = settlement_date(val_date, 2)

        settle = pd.Timestamp(settle)
        self.validate(bond, settle)

        pay_dates = coupon_schedule_after_settlement(settle, bond.maturity, bond.freq)
        coupon_cf = bond.face * (bond.coupon_rate / bond.freq)
        cfs = np.full(len(pay_dates), coupon_cf, dtype=float)
        cfs[-1] += bond.face

        # settlement-adjusted discounting
        dfs_pay = self.curve.df(pay_dates)
        df_settle = float(self.curve.df([settle])[0])
        dfs = dfs_pay / df_settle

        pv = float(np.sum(cfs * dfs))
        dirty = 100.0 * pv / bond.face

        ai_amt = accrued_interest(settle, bond.maturity, bond.coupon_rate, bond.face, bond.freq, bond.day_count)
        ai = 100.0 * ai_amt / bond.face

        clean = dirty - ai
        return dirty, clean, ai

In [50]:
pricer = BondPricer(curve)
print(pricer.price(bond, val_date, pd.Timestamp("2026-02-15")))
print(pricer.price(bond, val_date, pd.Timestamp("2026-02-16")))

(100.93844252553204, 100.93844252553204, 0.0)
(100.95315627905092, 100.93926739016203, 0.01388888888888889)


---

## Bond Pricing Engine Result: Time-Consistent Clean/Dirty Bond Pricer

We implemented a fixed-rate bond pricing engine with:
- Semiannual coupon schedule generation (maturity-anchored approximation)
- Accrued interest under day-count conventions (30/360, ACT/365)
- Dirty and clean price computation
- Settlement-aware discounting via:
$$
D_s(T)=\frac{D(T)}{D(s)}
$$
which ensures that prices evolve consistently when settlement date changes.

### Production caveats (explicitly acknowledged)
- Coupon schedules in production depend on issue date, first coupon date, and calendars (stubs, EOM rules).
- Settlement is simplified as T+2 calendar days (production uses business-day calendars + holidays).
- Curve conventions must match bond conventions; in production we share one cashflow engine for both curve calibration and bond pricing.

---

# **III. Portfolio Pricing (Vectorized) and Quality Control Flags**

## Goal
Price a portfolio (≥ 20 bonds) efficiently and safely:
- Compute dirty/clean/accrued for each bond
- Apply scenario-ready structure (we will reuse outputs for DV01 and shocks)
- Add QC flags for invalid inputs and suspicious outputs

## Why this matters in valuation teams
Valuation groups price *millions* of instruments daily. The hard part is not pricing one bond,
but pricing many while preventing bad inputs from corrupting downstream risk and P&L.

## QC flags we add
- `MATURED`: settlement ≥ maturity
- `BAD_COUPON`: coupon outside plausible bounds
- `BAD_FREQ`: unsupported frequency
- `BAD_DATES`: schedule not increasing
- `NEGATIVE_PRICE`: clean or dirty < 0
- `EXTREME_PRICE`: price outside loose bounds (e.g. < 20 or > 200 per 100)

In [51]:
# Build a portfolio of 20 bonds

def make_sample_portfolio(n: int = 20, seed: int = 7) -> pd.DataFrame:
    rng = np.random.default_rng(seed)

    # maturities between 1Y and 10Y from val_date
    mats = [val_date + pd.DateOffset(years=int(y)) for y in rng.integers(1, 11, size=n)]
    mats = [pd.Timestamp(m).replace(month=2, day=15) for m in mats]  # align to Feb 15 cycle

    coupons = rng.uniform(0.02, 0.08, size=n)  # 2% to 8%
    freqs = rng.choice([2], size=n)            # semiannual baseline
    dcs = rng.choice(["30/360", "ACT/365"], size=n, p=[0.8, 0.2])

    df = pd.DataFrame({
        "bond_id": [f"BOND_{i:03d}" for i in range(n)],
        "maturity": mats,
        "coupon_rate": coupons,
        "freq": freqs,
        "day_count": dcs,
        "face": 100.0
    })
    return df

portfolio_df = make_sample_portfolio(20)
portfolio_df.head()

,bond_id,maturity,coupon_rate,freq,day_count,face
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0
1,BOND_001,2033-02-15,0.036706,2,30/360,100.0
2,BOND_002,2033-02-15,0.035292,2,30/360,100.0
3,BOND_003,2035-02-15,0.046705,2,30/360,100.0
4,BOND_004,2032-02-15,0.050273,2,30/360,100.0


In [52]:
# Pricing portfolio with QC flags (loop now, vectorize next)

def price_portfolio(pricer: BondPricer, portfolio: pd.DataFrame, val_date: pd.Timestamp, settle: pd.Timestamp) -> pd.DataFrame:
    rows = []
    for _, r in portfolio.iterrows():
        bond = Bond(
            bond_id=str(r["bond_id"]),
            maturity=pd.Timestamp(r["maturity"]),
            coupon_rate=float(r["coupon_rate"]),
            freq=int(r["freq"]),
            day_count=str(r["day_count"]),
            face=float(r["face"]),
        )

        flags = []
        try:
            dirty, clean, ai = pricer.price(bond, val_date, settle)
            if dirty < 0 or clean < 0:
                flags.append("NEGATIVE_PRICE")
            if dirty < 20 or dirty > 200 or clean < 20 or clean > 200:
                flags.append("EXTREME_PRICE")
        except Exception as e:
            dirty, clean, ai = np.nan, np.nan, np.nan
            flags.append(type(e).__name__)  # e.g., ValueError, NotImplementedError

        rows.append({
            "bond_id": bond.bond_id,
            "maturity": bond.maturity,
            "coupon_rate": bond.coupon_rate,
            "freq": bond.freq,
            "day_count": bond.day_count,
            "dirty": dirty,
            "clean": clean,
            "accrued": ai,
            "flags": "|".join(flags) if flags else ""
        })

    return pd.DataFrame(rows)

settle = pd.Timestamp("2026-02-16")  # use a non-coupon day to see accrued > 0 on many bonds
pricer = BondPricer(curve)

priced = price_portfolio(pricer, portfolio_df, val_date, settle)
priced.head(10)

,bond_id,maturity,coupon_rate,freq,day_count,dirty,clean,accrued,flags
0,BOND_000,2036-02-15,0.038182,2,30/360,94.504154,94.493547,0.010606,
1,BOND_001,2033-02-15,0.036706,2,30/360,94.369914,94.359718,0.010196,
2,BOND_002,2033-02-15,0.035292,2,30/360,93.539742,93.529938,0.009803,
3,BOND_003,2035-02-15,0.046705,2,30/360,100.923368,100.910395,0.012973,
4,BOND_004,2032-02-15,0.050273,2,30/360,101.723256,101.709291,0.013965,
5,BOND_005,2034-02-15,0.053210,2,30/360,104.870534,104.855754,0.014781,
6,BOND_006,2035-02-15,0.079730,2,30/360,124.890127,124.867980,0.022147,
7,BOND_007,2029-02-15,0.067560,2,30/360,104.305304,104.286537,0.018767,
8,BOND_008,2027-02-15,0.057331,2,ACT/365,100.844864,100.829027,0.015837,
9,BOND_009,2030-02-15,0.079338,2,30/360,110.717796,110.695757,0.022038,


In [53]:
ok = priced[priced["flags"] == ""]
print(len(priced), "total,", len(ok), "ok")
print(ok["clean"].min(), ok["clean"].max())

20 total, 20 ok
88.51152546462357 124.86797996182032


In [54]:
# QC function that produces explicit flags

def qc_flags_for_bond(bond: Bond, settle: pd.Timestamp) -> List[str]:
    flags = []

    if settle >= bond.maturity:
        flags.append("MATURED")

    if bond.freq not in (1, 2, 4):
        flags.append("BAD_FREQ")

    if bond.coupon_rate < -0.01 or bond.coupon_rate > 0.25:
        flags.append("BAD_COUPON")

    # schedule checks (only if not matured)
    if "MATURED" not in flags:
        dates = coupon_schedule_after_settlement(settle, bond.maturity, bond.freq)
        if len(dates) == 0:
            flags.append("NO_CASHFLOWS")
        else:
            if dates[-1] != pd.Timestamp(bond.maturity):
                flags.append("BAD_DATES")
            if any(dates[i] >= dates[i+1] for i in range(len(dates)-1)):
                flags.append("BAD_DATES")

    return flags

In [55]:
# Schedule cache
## Most portfolios have repeated maturity dates in reality
## So we cache schedules by (settle, maturity, freq).

from functools import lru_cache

@lru_cache(maxsize=100_000)
def cached_schedule(settle: pd.Timestamp, maturity: pd.Timestamp, freq: int) -> Tuple[pd.Timestamp, ...]:
    # lru_cache needs hashable objects; Timestamp is hashable
    dates = coupon_schedule_after_settlement(pd.Timestamp(settle), pd.Timestamp(maturity), int(freq))
    return tuple(dates)

---

## Vectorized Portfolio Pricing Strategy

Instead of pricing each bond separately:
1. Expand the portfolio into a cashflow table (one row per cashflow).
2. Call the curve discount factor function once for all unique payment dates.
3. Compute PVs in a vectorized way and aggregate by bond.

This pattern scales because:
- It reduces Python loops.
- It amortizes curve interpolation cost.
- It makes QC and scenario shocks easy (modify $D(t)$ or cashflows and recompute).

In [56]:
# Build Cashflow table

def build_cashflow_table(portfolio: pd.DataFrame, settle: pd.Timestamp) -> pd.DataFrame:
    rows = []
    for _, r in portfolio.iterrows():
        bond_id = str(r["bond_id"])
        maturity = pd.Timestamp(r["maturity"])
        c = float(r["coupon_rate"])
        freq = int(r["freq"])
        face = float(r["face"])

        # basic QC: skip matured here; flag later
        if settle >= maturity:
            continue

        pay_dates = cached_schedule(settle, maturity, freq)
        if len(pay_dates) == 0:
            continue

        coupon_cf = face * (c / freq)
        for i, d in enumerate(pay_dates):
            cf = coupon_cf
            if i == len(pay_dates) - 1:
                cf += face
            rows.append((bond_id, maturity, c, freq, str(r["day_count"]), face, pd.Timestamp(d), cf))

    return pd.DataFrame(
        rows,
        columns=["bond_id", "maturity", "coupon_rate", "freq", "day_count", "face", "pay_date", "cashflow"]
    )

cf_table = build_cashflow_table(portfolio_df, settle)
cf_table.head(10)

,bond_id,maturity,coupon_rate,freq,day_count,face,pay_date,cashflow
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2026-08-15,1.909097
1,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2027-02-15,1.909097
2,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2027-08-15,1.909097
3,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2028-02-15,1.909097
4,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2028-08-15,1.909097
5,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2029-02-15,1.909097
6,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2029-08-15,1.909097
7,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2030-02-15,1.909097
8,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2030-08-15,1.909097
9,BOND_000,2036-02-15,0.038182,2,30/360,100.0,2031-02-15,1.909097


In [57]:
# Vectorized PV + dirty price (settlement-adjusted)

def price_portfolio_vectorized(curve: ZeroCurve, portfolio: pd.DataFrame, val_date: pd.Timestamp, settle: pd.Timestamp) -> pd.DataFrame:
    settle = pd.Timestamp(settle)

    # Cashflow expansion
    cf = build_cashflow_table(portfolio, settle)
    if cf.empty:
        raise ValueError("Cashflow table is empty. Check portfolio or settlement/maturity logic.")

    # Discount factors for all unique pay dates in one shot
    unique_dates = sorted(cf["pay_date"].unique())
    df_pay = curve.df(unique_dates)
    df_map = dict(zip(unique_dates, df_pay))

    # Settlement DF for adjustment
    df_settle = float(curve.df([settle])[0])

    # Map D(T) and compute settlement-adjusted DF
    cf["df_pay"] = cf["pay_date"].map(df_map).astype(float)
    cf["df_settle_adj"] = cf["df_pay"] / df_settle
    cf["pv_cf"] = cf["cashflow"] * cf["df_settle_adj"]

    # Aggregate PV per bond
    pv_by_bond = cf.groupby("bond_id", as_index=False)["pv_cf"].sum().rename(columns={"pv_cf": "pv"})

    # Merge back bond static info (one row per bond)
    static_cols = ["bond_id", "maturity", "coupon_rate", "freq", "day_count", "face"]
    out = portfolio[static_cols].merge(pv_by_bond, on="bond_id", how="left")

    # Dirty per 100
    out["dirty"] = 100.0 * out["pv"] / out["face"]

    # Accrued per 100 (still per bond; keep as loop for correctness, optimize later)
    accrued_list = []
    flag_list = []
    for _, r in out.iterrows():
        bond = Bond(
            bond_id=str(r["bond_id"]),
            maturity=pd.Timestamp(r["maturity"]),
            coupon_rate=float(r["coupon_rate"]),
            freq=int(r["freq"]),
            day_count=str(r["day_count"]),
            face=float(r["face"]),
        )

        flags = qc_flags_for_bond(bond, settle)

        if "MATURED" in flags:
            accrued_list.append(np.nan)
        else:
            ai_amt = accrued_interest(settle, bond.maturity, bond.coupon_rate, bond.face, bond.freq, bond.day_count)
            accrued_list.append(100.0 * ai_amt / bond.face)

        flag_list.append("|".join(flags) if flags else "")

    out["accrued"] = accrued_list
    out["clean"] = out["dirty"] - out["accrued"]
    out["flags"] = flag_list

    # Additional QC flags based on outputs
    out.loc[out["dirty"] < 0, "flags"] = out["flags"].where(out["flags"] != "", "") + "|NEGATIVE_PRICE"
    out.loc[out["dirty"] > 200, "flags"] = out["flags"].where(out["flags"] != "", "") + "|EXTREME_PRICE"
    out["flags"] = out["flags"].str.strip("|")

    return out

priced_vec = price_portfolio_vectorized(curve, portfolio_df, val_date, settle)
priced_vec.head(10)

,bond_id,maturity,coupon_rate,freq,day_count,face,pv,dirty,accrued,clean,flags
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0,94.504154,94.504154,0.010606,94.493547,
1,BOND_001,2033-02-15,0.036706,2,30/360,100.0,94.369914,94.369914,0.010196,94.359718,
2,BOND_002,2033-02-15,0.035292,2,30/360,100.0,93.539742,93.539742,0.009803,93.529938,
3,BOND_003,2035-02-15,0.046705,2,30/360,100.0,100.923368,100.923368,0.012973,100.910395,
4,BOND_004,2032-02-15,0.050273,2,30/360,100.0,101.723256,101.723256,0.013965,101.709291,
5,BOND_005,2034-02-15,0.053210,2,30/360,100.0,104.870534,104.870534,0.014781,104.855754,
6,BOND_006,2035-02-15,0.079730,2,30/360,100.0,124.890127,124.890127,0.022147,124.867980,
7,BOND_007,2029-02-15,0.067560,2,30/360,100.0,104.305304,104.305304,0.018767,104.286537,
8,BOND_008,2027-02-15,0.057331,2,ACT/365,100.0,100.844864,100.844864,0.015837,100.829027,
9,BOND_009,2030-02-15,0.079338,2,30/360,100.0,110.717796,110.717796,0.022038,110.695757,


In [58]:
# Compare loop pricing vs vectorized pricing on the first 10 bonds:

priced_loop = price_portfolio(pricer, portfolio_df, val_date, settle)

cmp = priced_loop.merge(
    priced_vec[["bond_id", "dirty", "clean", "accrued", "flags"]],
    on="bond_id",
    suffixes=("_loop", "_vec"),
)

cmp["dirty_diff"] = cmp["dirty_loop"] - cmp["dirty_vec"]
cmp["clean_diff"] = cmp["clean_loop"] - cmp["clean_vec"]
cmp[["bond_id", "dirty_diff", "clean_diff"]].head(10)

,bond_id,dirty_diff,clean_diff
0,BOND_000,0.000000e+00,0.000000e+00
1,BOND_001,0.000000e+00,0.000000e+00
2,BOND_002,0.000000e+00,0.000000e+00
3,BOND_003,0.000000e+00,0.000000e+00
4,BOND_004,0.000000e+00,0.000000e+00
5,BOND_005,-1.421085e-14,-1.421085e-14
6,BOND_006,0.000000e+00,0.000000e+00
7,BOND_007,0.000000e+00,0.000000e+00
8,BOND_008,0.000000e+00,0.000000e+00
9,BOND_009,0.000000e+00,0.000000e+00


# **IV. Risk Measures (Duration, DV01, Convexity)**

## Why This Matters
Valuation teams are not judged on price alone.
They must explain and control sensitivity of prices to:

- Parallel rate shifts
- Curve shape changes
- Spread shocks (later)
- Scenario stress testing

We implement:

1) Macaulay Duration
2) Modified Duration
3) DV01 (Dollar Value of 1bp)
4) Convexity
5) Finite-difference validation checks

---

## Definitions

Let settlement-adjusted discount factors be:
$$
D_s(T) = \frac{D(T)}{D(s)}
$$

Dirty price:
$$
P = \sum_i CF_i D_s(t_i)
$$

### Macaulay Duration
$$
D_{Mac} = \frac{1}{P}\sum_i t_i CF_i D_s(t_i)
$$

### Modified Duration
If yield is $y$:
$$
D_{mod} = \frac{D_{Mac}}{1 + \frac{y}{m}}
$$

But in a curve-based framework, we define duration via price sensitivity:
$$
D_{mod} = -\frac{1}{P}\frac{\partial P}{\partial y}
$$

### DV01
$$
DV01 = -\frac{\partial P}{\partial y} \cdot 0.0001
$$

### Convexity
$$
Conv = \frac{1}{P}\frac{\partial^2 P}{\partial y^2}
$$

We compute both analytical (cashflow-weighted) and finite-difference versions.

In [59]:
# Parallel curve shock function

def shocked_curve_parallel(curve: ZeroCurve, shift_bp: float) -> ZeroCurve:
    """
    Parallel shift in continuously-compounded zero rates.
    shift_bp in basis points (e.g., +1.0 for +1bp).
    """
    shift = shift_bp / 10000.0  # convert bp to decimal

    # Convert knot DFs to zero rates
    dates = pd.to_datetime(curve.knot_dates)
    dfs = np.exp(curve.knot_log_dfs)

    taus = np.array([
        yearfrac(curve.val_date, d, curve.zero_day_count)
        for d in dates
    ])

    zeros = -np.log(dfs) / taus

    # Apply parallel shift
    zeros_shifted = zeros + shift

    # Rebuild discount factors
    dfs_shifted = np.exp(-zeros_shifted * taus)

    logdfs_shifted = np.log(dfs_shifted)

    return ZeroCurve(
        val_date=curve.val_date,
        knot_dates=curve.knot_dates.copy(),
        knot_log_dfs=logdfs_shifted,
        zero_day_count=curve.zero_day_count,
    )

In [60]:
# DV01 via finite difference (vectorized)

def compute_portfolio_dv01(curve: ZeroCurve, portfolio: pd.DataFrame, val_date: pd.Timestamp, settle: pd.Timestamp):
    base = price_portfolio_vectorized(curve, portfolio, val_date, settle)

    shocked = shocked_curve_parallel(curve, shift_bp=1.0)
    shocked_prices = price_portfolio_vectorized(shocked, portfolio, val_date, settle)

    out = base[["bond_id", "dirty"]].merge(
        shocked_prices[["bond_id", "dirty"]],
        on="bond_id",
        suffixes=("_base", "_up1bp")
    )

    out["dv01"] = out["dirty_up1bp"] - out["dirty_base"]

    return out

dv01_df = compute_portfolio_dv01(curve, portfolio_df, val_date, settle)
dv01_df.head()

,bond_id,dirty_base,dirty_up1bp,dv01
0,BOND_000,94.504154,94.425087,-0.079067
1,BOND_001,94.369914,94.311254,-0.058660
2,BOND_002,93.539742,93.481378,-0.058364
3,BOND_003,100.923368,100.847944,-0.075424
4,BOND_004,101.723256,101.669695,-0.053561


In [61]:
print(dv01_df["dv01"].min(), dv01_df["dv01"].max())

-0.08624893020329694 -0.009667087959343235


In [62]:
# Add Convexity (Finite Difference)
## We will compute convexity using symmetric finite differences
## This measures curvature and validates duration linearity.

def compute_portfolio_convexity(curve: ZeroCurve, portfolio: pd.DataFrame, val_date: pd.Timestamp, settle: pd.Timestamp):
    base = price_portfolio_vectorized(curve, portfolio, val_date, settle)

    up = shocked_curve_parallel(curve, shift_bp=1.0)
    down = shocked_curve_parallel(curve, shift_bp=-1.0)

    price_up = price_portfolio_vectorized(up, portfolio, val_date, settle)
    price_down = price_portfolio_vectorized(down, portfolio, val_date, settle)

    out = base[["bond_id", "dirty"]].merge(
        price_up[["bond_id", "dirty"]],
        on="bond_id",
        suffixes=("_base", "_up")
    ).merge(
        price_down[["bond_id", "dirty"]],
        on="bond_id"
    )

    out = out.rename(columns={"dirty": "dirty_down"})

    h = 0.0001

    out["convexity"] = (
        out["dirty_up"] + out["dirty_down"] - 2 * out["dirty_base"]
    ) / (out["dirty_base"] * h**2)

    return out[["bond_id", "convexity"]]

conv_df = compute_portfolio_convexity(curve, portfolio_df, val_date, settle)
conv_df.head()

,bond_id,convexity
0,BOND_000,78.385251
1,BOND_001,41.673294
2,BOND_002,41.894041
3,BOND_003,62.760942
4,BOND_004,30.077598


In [63]:
conv_df["convexity"].min(), conv_df["convexity"].max()

(np.float64(0.9736964691923332), np.float64(78.38525120875688))

In [64]:
# Duration from DV01 (Consistency Check)

def duration_from_dv01(dv01_df: pd.DataFrame):
    df = dv01_df.copy()
    df["mod_duration"] = -df["dv01"] / (df["dirty_base"] * 0.0001)
    return df[["bond_id", "mod_duration"]]

dur_df = duration_from_dv01(dv01_df)
dur_df.head()

,bond_id,mod_duration
0,BOND_000,8.366467
1,BOND_001,6.215978
2,BOND_002,6.239485
3,BOND_003,7.473386
4,BOND_004,5.265350


---

# **V. Scenario Analysis (Rate, Curve Shape and Credit Spread)**

## Goal
Compute scenario P&L for each bond and the total portfolio under:
- Parallel rate shocks: ±25bp, ±50bp
- Steepener / flattener (curve shape changes)
- (Next step) Credit spread widening for corporate bonds

## Design principle
We do **not** re-implement pricing. We reuse:
- cashflow table expansion
- settlement-adjusted discounting
- curve object with interpolation policy

Scenario engine only modifies the curve (or spreads), then calls the same pricing function.

## P&L convention
We define:
$$
\Delta P = P_{\text{shocked}} - P_{\text{base}}
$$
Negative means loss for a long bond under rate increases.

In [65]:
# Hypothetical zero-rate shock builder

def curve_from_shifted_zeros(curve: ZeroCurve, shift_func) -> ZeroCurve:
    """
    Build a new curve by shifting continuously-compounded zero rates z(t)
    by shift_func(tau) (in decimal).
    """
    dates = pd.to_datetime(curve.knot_dates)
    dfs = np.exp(curve.knot_log_dfs)

    taus = np.array([yearfrac(curve.val_date, d, curve.zero_day_count) for d in dates], dtype=float)
    zeros = -np.log(dfs) / taus

    shifts = np.array([shift_func(t) for t in taus], dtype=float)
    zeros_shifted = zeros + shifts

    dfs_shifted = np.exp(-zeros_shifted * taus)
    logdfs_shifted = np.log(dfs_shifted)

    return ZeroCurve(
        val_date=curve.val_date,
        knot_dates=curve.knot_dates.copy(),
        knot_log_dfs=logdfs_shifted,
        zero_day_count=curve.zero_day_count,
    )

In [66]:
# scenario shock functions

def parallel_shift_bp(bp: float):
    s = bp / 10000.0
    return lambda tau: s

def steepener_shift_bp(bp: float, pivot: float = 2.0, long: float = 10.0):
    """
    Steepener: short end up +bp, long end down -bp.
    Piecewise linear between pivot and long.
    """
    A = bp / 10000.0

    def f(tau: float) -> float:
        if tau <= pivot:
            return +A
        if tau >= long:
            return -A
        w = (tau - pivot) / (long - pivot)
        return (1 - w) * (+A) + w * (-A)

    return f

def flattener_shift_bp(bp: float, pivot: float = 2.0, long: float = 10.0):
    """
    Flattener: short end down -bp, long end up +bp.
    """
    A = bp / 10000.0

    def f(tau: float) -> float:
        if tau <= pivot:
            return -A
        if tau >= long:
            return +A
        w = (tau - pivot) / (long - pivot)
        return (1 - w) * (-A) + w * (+A)

    return f

In [67]:
# Scenario runner (vectorized)
## scenario pricing + P&L table

def run_rate_scenarios(
    curve: ZeroCurve,
    portfolio: pd.DataFrame,
    val_date: pd.Timestamp,
    settle: pd.Timestamp,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      - per-bond scenario table (base + shocked prices + P&L)
      - portfolio summary (total P&L per scenario)
    """
    base = price_portfolio_vectorized(curve, portfolio, val_date, settle)[["bond_id", "dirty"]].rename(columns={"dirty": "base"})

    scenarios = {
        "PAR_-50bp": curve_from_shifted_zeros(curve, parallel_shift_bp(-50)),
        "PAR_-25bp": curve_from_shifted_zeros(curve, parallel_shift_bp(-25)),
        "PAR_+25bp": curve_from_shifted_zeros(curve, parallel_shift_bp(+25)),
        "PAR_+50bp": curve_from_shifted_zeros(curve, parallel_shift_bp(+50)),
        "STEEPENER_25bp": curve_from_shifted_zeros(curve, steepener_shift_bp(25)),
        "FLATTENER_25bp": curve_from_shifted_zeros(curve, flattener_shift_bp(25)),
    }

    per_bond = base.copy()

    for name, scurve in scenarios.items():
        px = price_portfolio_vectorized(scurve, portfolio, val_date, settle)[["bond_id", "dirty"]].rename(columns={"dirty": name})
        per_bond = per_bond.merge(px, on="bond_id", how="left")
        per_bond[name + "_PnL"] = per_bond[name] - per_bond["base"]

    # Portfolio totals
    pnl_cols = [c for c in per_bond.columns if c.endswith("_PnL")]
    summary = pd.DataFrame({
        "scenario": pnl_cols,
        "total_pnl_per_100_notional": [per_bond[c].sum() for c in pnl_cols],
    })

    return per_bond, summary

per_bond_scen, scen_summary = run_rate_scenarios(curve, portfolio_df, val_date, settle)
scen_summary

,scenario,total_pnl_per_100_notional
0,PAR_-50bp_PnL,52.431716
1,PAR_-25bp_PnL,25.989558
2,PAR_+25bp_PnL,-25.545758
3,PAR_+50bp_PnL,-50.656334
4,STEEPENER_25bp_PnL,8.904997
5,FLATTENER_25bp_PnL,-8.653248


In [69]:
# Exposure summary (DV01 approximation vs scenario)

# Compare +25bp scenario vs DV01 linear approx
dv01_df = compute_portfolio_dv01(curve, portfolio_df, val_date, settle)

tmp = per_bond_scen[["bond_id", "PAR_+25bp_PnL"]].merge(dv01_df[["bond_id", "dv01"]], on="bond_id")
tmp["linear_25bp"] = 25.0 * tmp["dv01"]
tmp["nonlinear_diff"] = tmp["PAR_+25bp_PnL"] - tmp["linear_25bp"]

display(tmp.head(10), tmp["nonlinear_diff"].describe())

,bond_id,PAR_+25bp_PnL,dv01,linear_25bp,nonlinear_diff
0,BOND_000,-1.954627,-0.079067,-1.976665,0.022038
1,BOND_001,-1.454775,-0.058660,-1.466503,0.011728
2,BOND_002,-1.447413,-0.058364,-1.459100,0.011687
3,BOND_003,-1.866738,-0.075424,-1.885598,0.018860
4,BOND_004,-1.329889,-0.053561,-1.339021,0.009132
5,BOND_005,-1.733014,-0.069943,-1.748584,0.015570
6,BOND_006,-2.131881,-0.086106,-2.152641,0.020761
7,BOND_007,-0.719858,-0.028895,-0.722371,0.002513
8,BOND_008,-0.247591,-0.009915,-0.247885,0.000294
9,BOND_009,-0.973753,-0.039128,-0.978209,0.004456


count    20.000000
mean      0.010579
std       0.008128
min       0.000288
25%       0.002487
50%       0.010410
75%       0.018122
max       0.023448
Name: nonlinear_diff, dtype: float64

---

# Results — Scenario Analysis Results & Risk Interpretation

## 1. Parallel Rate Shock Sanity Check

Portfolio P&L (per 100 notional):

| Scenario | Total P&L |
|-----------|------------|
| -50bp     | +52.43     |
| -25bp     | +25.99     |
| +25bp     | -25.55     |
| +50bp     | -50.66     |

### Interpretation

1. **Directional consistency**
   - Rate decreases → portfolio gains
   - Rate increases → portfolio losses  
   This confirms the portfolio is net **long duration**.

2. **Near symmetry**
   - Loss at +50bp ≈ -50.66  
   - Gain at -50bp ≈ +52.43  

   The magnitudes are close but not identical.

3. **Convexity effect**
   The asymmetry arises from convexity:

   $$
   \Delta P \approx -D \Delta y + \frac{1}{2} C (\Delta y)^2
   $$

   Because convexity $C > 0$ for standard fixed-rate bonds:
   - Gains from rate decreases are slightly larger
   - Losses from rate increases are slightly smaller than linear duration prediction

   This is expected and confirms second-order risk behavior is working correctly.

---

## 2. Linear Approximation vs True Scenario

For +25bp:

$$
\Delta P_{\text{linear}} \approx 25 \times DV01
$$

Observed nonlinear adjustment statistics:

- Mean convexity adjustment ≈ 0.0106
- Max ≈ 0.0234

### Interpretation

The difference:

$$
\Delta P_{\text{true}} - 25 \cdot DV01 > 0
$$

This means the linear approximation slightly **overstates the loss**, which is consistent with positive convexity.

If this sign were reversed, it would indicate:
- Incorrect convexity implementation
- Numerical instability
- Discounting inconsistency

The current behavior is correct and internally consistent.

---

## 3. Curve Shape Scenarios

| Scenario          | Total P&L |
|-------------------|------------|
| Steepener +25bp   | +8.90      |
| Flattener +25bp   | -8.65      |

### Interpretation

Steepener logic:
- Short end ↑
- Long end ↓

Because the portfolio contains a large concentration of long-maturity bonds (2033–2036 range), it benefits more from long-end declines than it loses from short-end increases.

Flattener produces the opposite effect.

This confirms:
- Curve tilt mechanics are functioning
- Portfolio duration distribution is realistic
- Risk exposure is maturity-sensitive, not purely parallel

---

## 4. Validation Checks

The scenario engine passes the following internal consistency checks:

1. Directionally correct P&L  
2. Convexity asymmetry present  
3. Linear vs nonlinear reconciliation consistent  
4. Curve shape shocks produce maturity-sensitive effects  

---

---

# **VI. Key Rate Duration (Institutional Implementation)**

Parallel DV01 measures total curve sensitivity.

Key Rate Duration decomposes that sensitivity into localized maturity buckets.

Unlike parallel shifts, key rate shocks must be:

- Localized
- Smooth
- Independent of interpolation artifacts

We implement triangular bucket shocks centered at key maturities.

In [70]:
# Triangular Key Rate Shock

def triangular_key_rate_shift(curve: ZeroCurve, key_index: int, bp: float):
    """
    Apply triangular zero-rate bump centered at knot index key_index.
    """
    s = bp / 10000.0

    dates = pd.to_datetime(curve.knot_dates)
    dfs = np.exp(curve.knot_log_dfs)

    taus = np.array([yearfrac(curve.val_date, d, curve.zero_day_count) for d in dates])
    zeros = -np.log(dfs) / taus

    zeros_shifted = zeros.copy()

    for i in range(len(taus)):
        if i == key_index:
            zeros_shifted[i] += s
        elif i == key_index - 1:
            zeros_shifted[i] += 0.5 * s
        elif i == key_index + 1:
            zeros_shifted[i] += 0.5 * s

    dfs_shifted = np.exp(-zeros_shifted * taus)
    logdfs_shifted = np.log(dfs_shifted)

    return ZeroCurve(
        curve.val_date,
        curve.knot_dates.copy(),
        logdfs_shifted,
        curve.zero_day_count,
    )

In [71]:
# Compute Bucketed KRD

def compute_key_rate_duration(curve, portfolio, val_date, settle):
    base = price_portfolio_vectorized(curve, portfolio, val_date, settle)[["bond_id", "dirty"]]
    base = base.rename(columns={"dirty": "base"})

    results = []

    for idx in range(len(curve.knot_dates)):
        shifted_curve = triangular_key_rate_shift(curve, idx, 1.0)
        shocked = price_portfolio_vectorized(shifted_curve, portfolio, val_date, settle)[["bond_id", "dirty"]]
        shocked = shocked.rename(columns={"dirty": "shocked"})

        tmp = base.merge(shocked, on="bond_id")
        tmp["krd"] = tmp["shocked"] - tmp["base"]
        tmp["bucket_index"] = idx

        results.append(tmp[["bond_id", "bucket_index", "krd"]])

    return pd.concat(results, ignore_index=True)

In [72]:
krd_df = compute_key_rate_duration(curve, portfolio_df, val_date, settle)

portfolio_krd = krd_df.groupby("bucket_index")["krd"].sum()
portfolio_krd

bucket_index
0    0.001671
1    0.000835
2   -0.001198
3   -0.015221
4   -0.073500
5   -0.282506
6   -0.678380
7   -0.734307
Name: krd, dtype: float64

In [73]:
portfolio_krd.sum()

np.float64(-1.782607763975264)

In [74]:
parallel_1bp_curve = curve_from_shifted_zeros(curve, parallel_shift_bp(1.0))
base = price_portfolio_vectorized(curve, portfolio_df, val_date, settle)["dirty"].sum()
shocked = price_portfolio_vectorized(parallel_1bp_curve, portfolio_df, val_date, settle)["dirty"].sum()

print("Parallel 1bp DV01:", shocked - base)

Parallel 1bp DV01: -1.0302933377672616


## Key Rate Duration (KRD) — Partition-of-Unity Basis

Naively bumping individual knots causes overlap and double-counting, so bucket DV01s will not reconcile to parallel DV01.

We implement piecewise-linear "hat" basis functions over knot maturities. These basis functions satisfy:

$$
\sum_k b_k(\tau) = 1
$$

Therefore, the sum of bucket DV01s approximately equals the parallel DV01:

$$
\sum_k KRD_k \approx DV01_{\text{parallel}}
$$

This reconciliation is a critical institutional validation check.

In [76]:
# hat basis function

def hat_basis(taus: np.ndarray, k: int):
    """
    Returns a function b_k(tau) for piecewise-linear hat basis on taus grid.
    """
    n = len(taus)
    if not (0 <= k < n):
        raise ValueError("k out of range")

    def b(tau: float) -> float:
        if k == 0:
            if tau <= taus[0]:
                return 1.0
            if tau >= taus[1]:
                return 0.0
            return (taus[1] - tau) / (taus[1] - taus[0])

        if k == n - 1:
            if tau <= taus[n - 2]:
                return 0.0
            if tau >= taus[n - 1]:
                return 1.0
            return (tau - taus[n - 2]) / (taus[n - 1] - taus[n - 2])

        # interior
        if tau <= taus[k - 1] or tau >= taus[k + 1]:
            return 0.0
        if tau <= taus[k]:
            return (tau - taus[k - 1]) / (taus[k] - taus[k - 1])
        else:
            return (taus[k + 1] - tau) / (taus[k + 1] - taus[k])

    return b

In [77]:
# apply KRD bump using basis

def key_rate_curve_hat(curve: ZeroCurve, k: int, bp: float) -> ZeroCurve:
    """
    Apply +bp shock to zero rates using hat basis b_k(tau).
    """
    s = bp / 10000.0

    dates = pd.to_datetime(curve.knot_dates)
    dfs = np.exp(curve.knot_log_dfs)

    taus = np.array([yearfrac(curve.val_date, d, curve.zero_day_count) for d in dates], dtype=float)
    zeros = -np.log(dfs) / taus

    b_k = hat_basis(taus, k)
    shifts = np.array([b_k(t) for t in taus], dtype=float) * s

    zeros_shifted = zeros + shifts

    dfs_shifted = np.exp(-zeros_shifted * taus)
    logdfs_shifted = np.log(dfs_shifted)

    return ZeroCurve(curve.val_date, curve.knot_dates.copy(), logdfs_shifted, curve.zero_day_count)

In [79]:
# compute KRD table + reconciliation

def compute_krd_hat(curve: ZeroCurve, portfolio: pd.DataFrame, val_date: pd.Timestamp, settle: pd.Timestamp, bp: float = 1.0):
    base_total = price_portfolio_vectorized(curve, portfolio, val_date, settle)["dirty"].sum()

    bucket_pnl = []
    for k in range(len(curve.knot_dates)):
        scurve = key_rate_curve_hat(curve, k, bp)
        shocked_total = price_portfolio_vectorized(scurve, portfolio, val_date, settle)["dirty"].sum()
        bucket_pnl.append(shocked_total - base_total)

    bucket_pnl = np.array(bucket_pnl, dtype=float)

    # parallel DV01 for the same bp
    par_curve = curve_from_shifted_zeros(curve, parallel_shift_bp(bp))
    par_total = price_portfolio_vectorized(par_curve, portfolio, val_date, settle)["dirty"].sum()
    par_dv01 = par_total - base_total

    return bucket_pnl, par_dv01

bucket_pnl, par_dv01 = compute_krd_hat(curve, portfolio_df, val_date, settle, bp=1.0)

print("Bucket sum:", bucket_pnl.sum())
print("Parallel DV01:", par_dv01)
print("Difference:", bucket_pnl.sum() - par_dv01)

Bucket sum: -1.0303758845650464
Parallel DV01: -1.0302933377672616
Difference: -8.254679778474383e-05


---

# Key Rate Duration Results (KRD) (Bucketed DV01)

## 1. Why KRD exists
Parallel DV01 measures total sensitivity to a uniform shift in rates.  
Key Rate Duration decomposes that exposure into maturity buckets, which is essential for:
- hedging (choosing specific tenors)
- understanding curve-shape risk
- risk reporting and stress testing

## 2. Implementation detail (institutional requirement)
Naively bumping individual knots can **double-count** exposure and fails to reconcile to parallel DV01.

We therefore used piecewise-linear "hat" basis functions $b_k(\tau)$ defined on the curve knot maturities such that:

$$
\sum_k b_k(\tau) = 1 \quad \text{(partition of unity)}
$$

A +1bp key-rate shock is implemented as a localized bump in continuously-compounded zero rates:

$$
z'(\tau) = z(\tau) + 0.0001 \cdot b_k(\tau)
$$

This preserves curve structure and avoids arbitrary interpolation artifacts.

## 3. Critical validation: KRD must reconcile to parallel DV01
We validated:

- Sum of bucket P&Ls under +1bp bumps:
  $$\sum_k KRD_k = -1.030376$$
- Portfolio parallel DV01 under +1bp:
  $$DV01_{parallel} = -1.030293$$

Difference:
$$
\sum_k KRD_k - DV01_{parallel} \approx -8.25 \times 10^{-5}
$$

This is within numerical tolerance and confirms the KRD decomposition is consistent.

## 4. Interpretation
- Buckets with larger negative values contribute more to portfolio rate risk.
- Concentration of negative bucket DV01 at long maturities implies the portfolio is long duration and long-end exposed.
- This bucket decomposition is the foundation for tenor hedging (e.g., hedging with 5Y vs 10Y vs long bond exposures).


In [80]:
# exposure table

# Build readable bucket table
knot_dates = pd.to_datetime(curve.knot_dates)
knot_taus = np.array([yearfrac(curve.val_date, d, curve.zero_day_count) for d in knot_dates], dtype=float)

bucket_table = pd.DataFrame({
    "bucket_index": np.arange(len(bucket_pnl)),
    "knot_date": knot_dates,
    "tau_years": knot_taus,
    "bucket_dv01": bucket_pnl,  # per 100 notional for +1bp localized bucket shock
})

bucket_table["bucket_dv01_pct_of_total"] = bucket_table["bucket_dv01"] / bucket_table["bucket_dv01"].sum()
bucket_table

,bucket_index,knot_date,tau_years,bucket_dv01,bucket_dv01_pct_of_total
0,0,2026-02-20,0.019178,0.001671,-0.001621
1,1,2026-03-13,0.076712,0.000000,-0.000000
2,2,2026-05-15,0.249315,0.000000,-0.000000
3,3,2026-08-14,0.498630,-0.002396,0.002326
4,4,2027-02-12,0.997260,-0.025649,0.024893
5,5,2028-02-15,2.005479,-0.093303,0.090553
6,6,2031-02-15,5.008219,-0.352736,0.342337
7,7,2036-02-15,10.010959,-0.557962,0.541513


In [81]:
# Show concentration

bucket_table.sort_values("bucket_dv01").reset_index(drop=True)

,bucket_index,knot_date,tau_years,bucket_dv01,bucket_dv01_pct_of_total
0,7,2036-02-15,10.010959,-0.557962,0.541513
1,6,2031-02-15,5.008219,-0.352736,0.342337
2,5,2028-02-15,2.005479,-0.093303,0.090553
3,4,2027-02-12,0.997260,-0.025649,0.024893
4,3,2026-08-14,0.498630,-0.002396,0.002326
5,1,2026-03-13,0.076712,0.000000,-0.000000
6,2,2026-05-15,0.249315,0.000000,-0.000000
7,0,2026-02-20,0.019178,0.001671,-0.001621


# **VII. Corporate Bond Pricing (Spread Over Risk-Free Curve)**

## Objective
Extend the pricing engine to value corporate bonds by discounting cashflows using:
- the risk-free discount curve $D(t)$
- a constant credit spread $s$ applied to **all** cashflows (Z-spread style)

## Why spreads are applied to all cashflows
A corporate bond is a stream of risky cashflows. Each cashflow is subject to:
- default risk
- liquidity risk
- funding/valuation adjustments (in production)

Therefore spread is applied to every cashflow date $t_i$:

## Spread-adjusted discount factor
Using settlement-adjusted discounting and a constant spread $s$ (in decimal):

$$
D_{corp}(t_i) = \frac{D(t_i)}{D(s)} \cdot e^{-s \cdot \tau(s, t_i)}
$$

where $\tau(s,t_i)$ is the year fraction from settlement to the cashflow date under the curve day count basis.

## Interpretation
- This is the standard "discount margin" / Z-spread intuition.
- In practice, OAS extends this by using a stochastic rate model and subtracting the embedded option value.
  Here we implement the deterministic baseline first.

In [82]:
# Corporate bond definition

@dataclass(frozen=True)
class CorporateBond(Bond):
    spread: float = 0.0  # constant credit spread in decimal (e.g. 0.015 = 150bp)

In [83]:
# Spread-adjusted pricing function (vectorized-friendly)

def price_corporate_bond_dirty_clean(
    curve: ZeroCurve,
    bond: CorporateBond,
    val_date: pd.Timestamp,
    settle: Optional[pd.Timestamp] = None,
) -> Tuple[float, float, float]:
    """
    Corporate bond pricing with constant spread applied to ALL cashflows:
      D_corp(t) = D(t)/D(settle) * exp(-spread * tau(settle,t))

    Returns (dirty, clean, accrued) per 100.
    """
    if settle is None:
        settle = settlement_date(val_date, 2)

    settle = pd.Timestamp(settle)
    if settle >= bond.maturity:
        raise ValueError(f"{bond.bond_id}: matured at settlement.")

    if bond.spread < -0.001:
        raise ValueError("Suspicious negative spread.")

    pay_dates = coupon_schedule_after_settlement(settle, bond.maturity, bond.freq)
    coupon_cf = bond.face * (bond.coupon_rate / bond.freq)

    cfs = np.full(len(pay_dates), coupon_cf, dtype=float)
    cfs[-1] += bond.face

    # base settlement-adjusted DFs
    df_pay = curve.df(pay_dates)
    df_settle = float(curve.df([settle])[0])
    df_settle_adj = df_pay / df_settle

    # spread adjustment applied to all cashflows
    taus_settle = np.array([yearfrac(settle, d, curve.zero_day_count) for d in pay_dates], dtype=float)
    spread_adj = np.exp(-bond.spread * taus_settle)

    dfs_corp = df_settle_adj * spread_adj

    pv = float(np.sum(cfs * dfs_corp))
    dirty = 100.0 * pv / bond.face

    ai_amt = accrued_interest(settle, bond.maturity, bond.coupon_rate, bond.face, bond.freq, bond.day_count)
    ai = 100.0 * ai_amt / bond.face
    clean = dirty - ai

    return dirty, clean, ai

In [84]:
# Demo: price a corporate bond at different spreads

corp = CorporateBond(
    bond_id="CORP_5Y_5PCT",
    maturity=pd.Timestamp("2031-02-15"),
    coupon_rate=0.05,
    freq=2,
    day_count="30/360",
    face=100.0,
    spread=0.0
)

settle = pd.Timestamp("2026-02-16")

for s_bp in [0, 50, 100, 200]:
    corp_s = CorporateBond(**{**corp.__dict__, "spread": s_bp/10000.0})
    dirty, clean, ai = price_corporate_bond_dirty_clean(curve, corp_s, val_date, settle)
    print(f"Spread {s_bp:>3}bp -> Dirty {dirty:.6f}, Clean {clean:.6f}, AI {ai:.6f}")

Spread   0bp -> Dirty 100.953156, Clean 100.939267, AI 0.013889
Spread  50bp -> Dirty 98.713017, Clean 98.699128, AI 0.013889
Spread 100bp -> Dirty 96.525972, Clean 96.512083, AI 0.013889
Spread 200bp -> Dirty 92.306058, Clean 92.292169, AI 0.013889


In [85]:
# Spread DV01 / Spread Duration

def spread_dv01(curve: ZeroCurve, bond: CorporateBond, val_date: pd.Timestamp, settle: pd.Timestamp) -> float:
    base_dirty, _, _ = price_corporate_bond_dirty_clean(curve, bond, val_date, settle)
    bumped = CorporateBond(**{**bond.__dict__, "spread": bond.spread + 1/10000.0})
    bumped_dirty, _, _ = price_corporate_bond_dirty_clean(curve, bumped, val_date, settle)
    return bumped_dirty - base_dirty

def spread_duration(curve: ZeroCurve, bond: CorporateBond, val_date: pd.Timestamp, settle: pd.Timestamp) -> float:
    base_dirty, _, _ = price_corporate_bond_dirty_clean(curve, bond, val_date, settle)
    sdv01 = spread_dv01(curve, bond, val_date, settle)
    return -sdv01 / (base_dirty * 0.0001)

corp_150 = CorporateBond(**{**corp.__dict__, "spread": 150/10000.0})
sdv01 = spread_dv01(curve, corp_150, val_date, settle)
sdur = spread_duration(curve, corp_150, val_date, settle)
print("Spread DV01:", sdv01, "Spread Duration:", sdur)

Spread DV01: -0.04218490904337102 Spread Duration: 4.46917892480038


---

## Corporate Portfolio Pricing at Scale

To scale corporate bond valuation, we reuse the same vectorized cashflow table approach:

Dirty price:
$$
P = \sum_i CF_i \cdot \frac{D(t_i)}{D(s)} \cdot e^{-s \cdot \tau(s,t_i)}
$$

Key observation:
- Risk-free discounting and cashflow generation are identical to the Treasury case.
- Credit spread enters only through a multiplicative factor $e^{-s\tau}$ applied to every cashflow.

This makes spread shocks extremely efficient:
- rate shocks modify $D(t)$
- spread shocks modify $s$
- combined shocks modify both

In [86]:
# Vectorized corporate portfolio pricing

def add_random_spreads(portfolio: pd.DataFrame, seed: int = 11) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    out = portfolio.copy()
    # IG-ish spreads: 30bp to 250bp
    out["spread"] = rng.uniform(30, 250, size=len(out)) / 10000.0
    return out

corp_portfolio_df = add_random_spreads(portfolio_df)
corp_portfolio_df.head()

,bond_id,maturity,coupon_rate,freq,day_count,face,spread
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829
1,BOND_001,2033-02-15,0.036706,2,30/360,100.0,0.013984
2,BOND_002,2033-02-15,0.035292,2,30/360,100.0,0.016233
3,BOND_003,2035-02-15,0.046705,2,30/360,100.0,0.003631
4,BOND_004,2032-02-15,0.050273,2,30/360,100.0,0.006254


In [87]:
# Corporate cashflow table (same as before, just keep spread)

def build_cashflow_table_corp(portfolio: pd.DataFrame, settle: pd.Timestamp) -> pd.DataFrame:
    rows = []
    for _, r in portfolio.iterrows():
        bond_id = str(r["bond_id"])
        maturity = pd.Timestamp(r["maturity"])
        c = float(r["coupon_rate"])
        freq = int(r["freq"])
        face = float(r["face"])
        dc = str(r["day_count"])
        spr = float(r["spread"])

        if settle >= maturity:
            continue

        pay_dates = cached_schedule(settle, maturity, freq)
        if len(pay_dates) == 0:
            continue

        coupon_cf = face * (c / freq)
        for i, d in enumerate(pay_dates):
            cf = coupon_cf
            if i == len(pay_dates) - 1:
                cf += face
            rows.append((bond_id, maturity, c, freq, dc, face, spr, pd.Timestamp(d), cf))

    return pd.DataFrame(
        rows,
        columns=["bond_id","maturity","coupon_rate","freq","day_count","face","spread","pay_date","cashflow"]
    )

cf_corp = build_cashflow_table_corp(corp_portfolio_df, settle)
cf_corp.head()

,bond_id,maturity,coupon_rate,freq,day_count,face,spread,pay_date,cashflow
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829,2026-08-15,1.909097
1,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829,2027-02-15,1.909097
2,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829,2027-08-15,1.909097
3,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829,2028-02-15,1.909097
4,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829,2028-08-15,1.909097


In [88]:
# Vectorized corporate pricing

def price_corporate_portfolio_vectorized(
    curve: ZeroCurve,
    portfolio: pd.DataFrame,
    val_date: pd.Timestamp,
    settle: pd.Timestamp,
) -> pd.DataFrame:
    settle = pd.Timestamp(settle)

    cf = build_cashflow_table_corp(portfolio, settle)
    if cf.empty:
        raise ValueError("Corporate cashflow table is empty.")

    # discount factors for all unique pay dates
    unique_dates = sorted(cf["pay_date"].unique())
    df_pay = curve.df(unique_dates)
    df_map = dict(zip(unique_dates, df_pay))

    df_settle = float(curve.df([settle])[0])

    cf["df_pay"] = cf["pay_date"].map(df_map).astype(float)
    cf["df_settle_adj"] = cf["df_pay"] / df_settle

    # tau from settle to pay_date for spread factor
    cf["tau_settle"] = cf["pay_date"].apply(lambda d: yearfrac(settle, d, curve.zero_day_count)).astype(float)
    cf["spread_adj"] = np.exp(-cf["spread"].astype(float) * cf["tau_settle"])

    cf["pv_cf"] = cf["cashflow"] * cf["df_settle_adj"] * cf["spread_adj"]

    pv_by_bond = cf.groupby("bond_id", as_index=False)["pv_cf"].sum().rename(columns={"pv_cf": "pv"})

    static_cols = ["bond_id","maturity","coupon_rate","freq","day_count","face","spread"]
    out = portfolio[static_cols].merge(pv_by_bond, on="bond_id", how="left")
    out["dirty"] = 100.0 * out["pv"] / out["face"]

    # accrued per 100
    accrued = []
    flags = []
    for _, r in out.iterrows():
        bond = Bond(
            bond_id=str(r["bond_id"]),
            maturity=pd.Timestamp(r["maturity"]),
            coupon_rate=float(r["coupon_rate"]),
            freq=int(r["freq"]),
            day_count=str(r["day_count"]),
            face=float(r["face"]),
        )
        f = qc_flags_for_bond(bond, settle)
        flags.append("|".join(f) if f else "")

        if "MATURED" in f:
            accrued.append(np.nan)
        else:
            ai_amt = accrued_interest(settle, bond.maturity, bond.coupon_rate, bond.face, bond.freq, bond.day_count)
            accrued.append(100.0 * ai_amt / bond.face)

    out["accrued"] = accrued
    out["clean"] = out["dirty"] - out["accrued"]
    out["flags"] = flags

    return out

In [89]:
corp_priced = price_corporate_portfolio_vectorized(curve, corp_portfolio_df, val_date, settle)
corp_priced.head(10)

,bond_id,maturity,coupon_rate,freq,day_count,face,spread,pv,dirty,accrued,clean,flags
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829,90.017060,90.017060,0.010606,90.006454,
1,BOND_001,2033-02-15,0.036706,2,30/360,100.0,0.013984,86.536609,86.536609,0.010196,86.526413,
2,BOND_002,2033-02-15,0.035292,2,30/360,100.0,0.016233,84.560044,84.560044,0.009803,84.550241,
3,BOND_003,2035-02-15,0.046705,2,30/360,100.0,0.003631,98.224779,98.224779,0.012973,98.211806,
4,BOND_004,2032-02-15,0.050273,2,30/360,100.0,0.006254,98.431521,98.431521,0.013965,98.417556,
5,BOND_005,2034-02-15,0.053210,2,30/360,100.0,0.023421,89.833634,89.833634,0.014781,89.818854,
6,BOND_006,2035-02-15,0.079730,2,30/360,100.0,0.004549,121.042595,121.042595,0.022147,121.020448,
7,BOND_007,2029-02-15,0.067560,2,30/360,100.0,0.005855,102.627568,102.627568,0.018767,102.608802,
8,BOND_008,2027-02-15,0.057331,2,ACT/365,100.0,0.023863,98.506347,98.506347,0.015837,98.490510,
9,BOND_009,2030-02-15,0.079338,2,30/360,100.0,0.016681,104.392322,104.392322,0.022038,104.370284,


In [90]:
# Spread scenario shocks (portfolio)

def run_spread_scenarios(
    curve: ZeroCurve,
    corp_portfolio: pd.DataFrame,
    val_date: pd.Timestamp,
    settle: pd.Timestamp,
):
    base = price_corporate_portfolio_vectorized(curve, corp_portfolio, val_date, settle)[["bond_id","dirty"]]
    base = base.rename(columns={"dirty":"base"})

    scenarios = {
        "SPR_+25bp": 25/10000.0,
        "SPR_+100bp": 100/10000.0,
    }

    per_bond = base.copy()

    for name, bump in scenarios.items():
        shocked_port = corp_portfolio.copy()
        shocked_port["spread"] = shocked_port["spread"] + bump

        shocked = price_corporate_portfolio_vectorized(curve, shocked_port, val_date, settle)[["bond_id","dirty"]]
        shocked = shocked.rename(columns={"dirty": name})

        per_bond = per_bond.merge(shocked, on="bond_id")
        per_bond[name + "_PnL"] = per_bond[name] - per_bond["base"]

    summary = pd.DataFrame({
        "scenario": [c for c in per_bond.columns if c.endswith("_PnL")],
        "total_pnl_per_100_notional": [per_bond[c].sum() for c in per_bond.columns if c.endswith("_PnL")]
    })

    return per_bond, summary

spread_per_bond, spread_summary = run_spread_scenarios(curve, corp_portfolio_df, val_date, settle)
spread_summary

,scenario,total_pnl_per_100_notional
0,SPR_+25bp_PnL,-23.464322
1,SPR_+100bp_PnL,-91.516137


---

## Combined Rate + Spread Scenarios (Valuation Desk Style)

Corporate bond valuation is driven by two major risk channels:
1) Risk-free curve movements (rates)
2) Credit spread movements (spread widening/tightening)

We therefore report scenario P&L under combinations such as:
- Rates +50bp with spreads +25bp (risk-off shock)
- Rates -50bp with spreads -25bp (risk-on shock)

Mechanically:
- Rate shocks modify the risk-free curve $D(t)$
- Spread shocks modify the spread term $e^{-s \tau}$ applied to every cashflow

The pricing engine remains unchanged; only inputs are shocked.
This is critical for production infrastructure: one pricing pipeline, many scenario views.

In [91]:
# Combined scenario runner

def run_combined_rate_spread_scenarios(
    base_curve: ZeroCurve,
    corp_portfolio: pd.DataFrame,
    val_date: pd.Timestamp,
    settle: pd.Timestamp,
) -> pd.DataFrame:
    """
    Scenarios:
      - Parallel rate shocks: -50, -25, +25, +50 (bp)
      - Spread shocks: 0, +25, +100 (bp)
      - Combine them into a grid and report total P&L
    """
    settle = pd.Timestamp(settle)

    base_prices = price_corporate_portfolio_vectorized(base_curve, corp_portfolio, val_date, settle)
    base_total = base_prices["dirty"].sum()

    rate_shocks = [-50, -25, 0, 25, 50]
    spread_shocks = [0, 25, 100]

    rows = []
    for r_bp in rate_shocks:
        scurve = curve_from_shifted_zeros(base_curve, parallel_shift_bp(r_bp))

        for s_bp in spread_shocks:
            shocked_port = corp_portfolio.copy()
            shocked_port["spread"] = shocked_port["spread"] + s_bp / 10000.0

            shocked_prices = price_corporate_portfolio_vectorized(scurve, shocked_port, val_date, settle)
            shocked_total = shocked_prices["dirty"].sum()

            rows.append({
                "rate_shock_bp": r_bp,
                "spread_shock_bp": s_bp,
                "total_dirty_base": base_total,
                "total_dirty_shocked": shocked_total,
                "total_pnl_per_100_notional": shocked_total - base_total,
            })

    out = pd.DataFrame(rows)
    return out.sort_values(["rate_shock_bp", "spread_shock_bp"]).reset_index(drop=True)

combo = run_combined_rate_spread_scenarios(curve, corp_portfolio_df, val_date, settle)
combo

,rate_shock_bp,spread_shock_bp,total_dirty_base,total_dirty_shocked,total_pnl_per_100_notional
0,-50,0,1907.768073,1955.913753,4.814568e+01
1,-50,25,1907.768073,1931.635425,2.386735e+01
2,-50,100,1907.768073,1861.234675,-4.653340e+01
3,-25,0,1907.768073,1931.635425,2.386735e+01
4,-25,25,1907.768073,1907.768073,-2.273737e-13
5,-25,100,1907.768073,1838.553225,-6.921485e+01
6,0,0,1907.768073,1907.768073,0.000000e+00
7,0,25,1907.768073,1884.303751,-2.346432e+01
8,0,100,1907.768073,1816.251935,-9.151614e+01
9,25,0,1907.768073,1884.303751,-2.346432e+01


In [92]:
# Pivot table

pivot = combo.pivot(index="rate_shock_bp", columns="spread_shock_bp", values="total_pnl_per_100_notional")
pivot

spread_shock_bp,0,25,100
rate_shock_bp,,,
-50,48.145680,2.386735e+01,-46.533397
-25,23.867352,-2.273737e-13,-69.214847
0,0.000000,-2.346432e+01,-91.516137
25,-23.464322,-4.653340e+01,-113.444579
50,-46.533397,-6.921485e+01,-135.007333


In [93]:
# Portfolio spread DV01 (risk dashboard metric)

def portfolio_spread_dv01(curve: ZeroCurve, corp_portfolio: pd.DataFrame, val_date: pd.Timestamp, settle: pd.Timestamp) -> float:
    base = price_corporate_portfolio_vectorized(curve, corp_portfolio, val_date, settle)["dirty"].sum()

    bumped = corp_portfolio.copy()
    bumped["spread"] = bumped["spread"] + 1/10000.0

    shocked = price_corporate_portfolio_vectorized(curve, bumped, val_date, settle)["dirty"].sum()
    return shocked - base

psdv01 = portfolio_spread_dv01(curve, corp_portfolio_df, val_date, settle)
print("Portfolio Spread DV01 (per 1bp):", psdv01)
print("Implied 25bp P&L from linear approx:", 25*psdv01)
print("Actual 25bp P&L:", spread_summary.loc[spread_summary["scenario"]=="SPR_+25bp_PnL","total_pnl_per_100_notional"].iloc[0])

Portfolio Spread DV01 (per 1bp): -0.9462587331993291
Implied 25bp P&L from linear approx: -23.656468329983227
Actual 25bp P&L: -23.46432230466108


---

# Combined Rate + Spread Scenario Table

Our pivot:

| Rate \ Spread | 0bp    | +25bp  | +100bp  |
| ------------- | ------ | ------ | ------- |
| -50bp         | +48.15 | +23.87 | -46.53  |
| -25bp         | +23.87 | ~0     | -69.21  |
| 0             | 0      | -23.46 | -91.52  |
| +25bp         | -23.46 | -46.53 | -113.44 |
| +50bp         | -46.53 | -69.21 | -135.01 |

---

## Interpretation

### 1) Symmetry in rate-only direction

In the spread = 0 column:

* -50bp → +48.15
* +50bp → -46.53

That’s convexity asymmetry, just like in Step V.

---

### 2) Spread-only direction (rate = 0 row)

* +25bp → -23.46
* +100bp → -91.52

Linear check:

Spread DV01 ≈ -0.9463

Linear 25bp approx:
[
25 \times (-0.9463) = -23.66
]

Actual:
-23.46

That’s extremely close.

So:

* Spread duration implementation is correct
* Spread is applied to all cashflows
* Spread convexity effect is small for 25bp

That’s exactly what we expect.

---

### 3) Cross interaction

We notice that:

Rate +25bp & Spread +25bp → -46.53

Which is approximately:

[
-23.46 + (-23.46) ≈ -46.92
]

Actual: -46.53

Small nonlinear interaction from convexity.

That means:

* Rate and spread risk are approximately additive for small shocks.
* Second-order effects exist but are modest.

That is the expected risk behavior.

---

# **VIII. Portfolio Risk Dashboard**

This dashboard summarizes the portfolio’s:

- Market value
- Parallel DV01 (rate risk)
- Convexity
- Key Rate Duration (term structure exposure)
- Spread DV01 (credit risk)
- Top contributors to rate and spread risk

In [94]:
# Portoflio totals

# Market value
base_total = price_corporate_portfolio_vectorized(curve, corp_portfolio_df, val_date, settle)["dirty"].sum()

parallel_dv01 = par_dv01  # from earlier
spread_dv01_total = psdv01

print("Total Market Value (per 100 notional sum):", base_total)
print("Parallel DV01:", parallel_dv01)
print("Spread DV01:", spread_dv01_total)

Total Market Value (per 100 notional sum): 1907.7680728781722
Parallel DV01: -1.0302933377672616
Spread DV01: -0.9462587331993291


In [95]:
# Key Rate bucket exposure table

bucket_table["abs_exposure"] = bucket_table["bucket_dv01"].abs()
bucket_table.sort_values("abs_exposure", ascending=False)

,bucket_index,knot_date,tau_years,bucket_dv01,bucket_dv01_pct_of_total,abs_exposure
7,7,2036-02-15,10.010959,-0.557962,0.541513,0.557962
6,6,2031-02-15,5.008219,-0.352736,0.342337,0.352736
5,5,2028-02-15,2.005479,-0.093303,0.090553,0.093303
4,4,2027-02-12,0.997260,-0.025649,0.024893,0.025649
3,3,2026-08-14,0.498630,-0.002396,0.002326,0.002396
0,0,2026-02-20,0.019178,0.001671,-0.001621,0.001671
2,2,2026-05-15,0.249315,0.000000,-0.000000,0.000000
1,1,2026-03-13,0.076712,0.000000,-0.000000,0.000000


In [96]:
# Top 5 bonds by rate DV01

rate_risk = compute_portfolio_dv01(curve, portfolio_df, val_date, settle)
rate_risk_sorted = rate_risk.sort_values("dv01", key=lambda x: x.abs(), ascending=False)
rate_risk_sorted.head(5)

,bond_id,dirty_base,dirty_up1bp,dv01
12,BOND_012,109.187743,109.101494,-0.086249
6,BOND_006,124.890127,124.804022,-0.086106
0,BOND_000,94.504154,94.425087,-0.079067
15,BOND_015,103.963170,103.886391,-0.076779
17,BOND_017,119.227594,119.151886,-0.075708


In [97]:
# Top 5 bonds by spread DV01

def compute_spread_dv01_per_bond(curve, corp_portfolio, val_date, settle):
    base = price_corporate_portfolio_vectorized(curve, corp_portfolio, val_date, settle)[["bond_id","dirty"]]
    base = base.rename(columns={"dirty":"base"})

    bumped = corp_portfolio.copy()
    bumped["spread"] = bumped["spread"] + 1/10000.0

    shocked = price_corporate_portfolio_vectorized(curve, bumped, val_date, settle)[["bond_id","dirty"]]
    shocked = shocked.rename(columns={"dirty":"shocked"})

    tmp = base.merge(shocked, on="bond_id")
    tmp["spread_dv01"] = tmp["shocked"] - tmp["base"]
    return tmp

spread_risk = compute_spread_dv01_per_bond(curve, corp_portfolio_df, val_date, settle)
spread_risk.sort_values("spread_dv01", key=lambda x: x.abs(), ascending=False).head(5)

,bond_id,base,shocked,spread_dv01
6,BOND_006,121.042595,120.959599,-0.082996
0,BOND_000,90.017060,89.942189,-0.074870
12,BOND_012,95.168889,95.095391,-0.073498
3,BOND_003,98.224779,98.151618,-0.073161
17,BOND_017,108.961836,108.893607,-0.068229


---

# Portfolio Risk Analysis — Institutional Valuation Report

## 1. Portfolio Overview

### Total Market Value

Total dirty market value (per 100 notional sum):

$$
\text{MV} = 1907.77
$$

With 20 bonds in the portfolio, the average dirty price is approximately:

$$
\frac{1907.77}{20} \approx 95.39
$$

This is economically reasonable given the presence of positive credit spreads across the portfolio.

---

## 2. Aggregate Risk Metrics

### Parallel DV01 (Rate Risk)

$$
DV01_{\text{parallel}} = -1.0303
$$

Interpretation:

- A +1bp parallel upward shift in the risk-free curve reduces total portfolio value by approximately **1.03** per 100 notional.
- A +100bp rate shock would imply a linear loss of approximately:

$$
-1.0303 \times 100 \approx -103.03
$$

The sign and magnitude are consistent with a long-duration fixed-income portfolio.

---

### Spread DV01 (Credit Risk)

$$
DV01_{\text{spread}} = -0.9463
$$

Interpretation:

- A +1bp widening in credit spreads reduces portfolio value by approximately **0.95** per 100 notional.
- Spread risk magnitude is comparable to rate risk, indicating material exposure to credit conditions.

Linear 25bp check:

$$
25 \times (-0.9463) = -23.66
$$

Observed P&L under +25bp spread shock:

$$
-23.46
$$

The close match confirms:

- Correct application of spreads to **all cashflows**
- Proper settlement-adjusted spread discounting
- Minimal second-order (spread convexity) distortion for small shocks

---

## 3. Key Rate Duration (Term Structure Decomposition)

Key Rate Duration was computed using partition-of-unity hat basis functions to ensure:

$$
\sum_k KRD_k \approx DV01_{\text{parallel}}
$$

Reconciliation:

- Sum of bucket DV01s: −1.03038  
- Parallel DV01: −1.03029  
- Difference: ~8.3e−05 (numerical tolerance)

This confirms internal consistency.

### Bucket Exposure Breakdown

| Maturity | Tau (years) | Bucket DV01 |
|-----------|-------------|-------------|
| 2036 | 10.01 | −0.558 |
| 2031 | 5.01 | −0.353 |
| 2028 | 2.01 | −0.093 |
| 2027 | 1.00 | −0.026 |
| Short-end | <0.5 | ~0 |

### Interpretation

- Over 85% of total rate risk resides in the **5–10 year sector**.
- Short-end exposure is negligible.
- Portfolio is structurally long duration at the intermediate-to-long end.

This aligns with earlier steepener/flattener scenario behavior.

---

## 4. Concentration Risk — Top Contributors

### Top 5 Bonds by Rate DV01

Largest contributors to rate sensitivity:

- BOND_012: −0.086
- BOND_006: −0.086
- BOND_000: −0.079
- BOND_015: −0.077
- BOND_017: −0.076

These bonds likely:
- Have longer maturities
- Exhibit higher duration
- Dominate term-structure risk

---

### Top 5 Bonds by Spread DV01

Largest contributors to credit sensitivity:

- BOND_006: −0.083
- BOND_000: −0.075
- BOND_012: −0.073
- BOND_003: −0.073
- BOND_017: −0.068

Observation:

Several bonds dominate **both rate and spread risk**, indicating concentration of long-end credit exposure.

---

## 5. Combined Rate + Spread Scenario Interpretation

Selected combined scenarios:

| Rate Shock | Spread Shock | Total P&L |
|------------|--------------|-----------|
| -50bp | 0bp | +48.15 |
| 0bp | +25bp | -23.46 |
| +25bp | +25bp | -46.53 |
| +50bp | +100bp | -135.01 |

### Interpretation

- Rate and spread risks are approximately additive for small shocks.
- Risk-off scenarios (rates ↑ + spreads ↑) produce significant downside.
- Spread shocks dominate for large widening scenarios.
- Convexity introduces mild asymmetry in rate-only movements.

This behavior is consistent with institutional fixed-income risk models.

---

## 6. Structural Risk Observations

1. Rate and spread risk magnitudes are comparable.
2. Risk is heavily concentrated in intermediate-to-long maturities.
3. Portfolio is convexity-positive.
4. Scenario results reconcile with DV01 approximations.
5. Key Rate Duration correctly decomposes parallel DV01.

---

## Conclusion

The portfolio analytics framework now supports:

- Zero curve bootstrapping
- Settlement-aware clean/dirty pricing
- Vectorized portfolio valuation
- Parallel DV01 and convexity
- Key Rate Duration decomposition
- Corporate spread discounting (Z-spread style)
- Spread DV01
- Combined rate + spread stress scenarios
- Risk concentration reporting

The analytics are internally consistent, economically interpretable, and structured in a way consistent with institutional valuation infrastructure.